In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import clear_output

from tqdm import tqdm
from nba_api.stats.static import players, teams
from nba_api.stats.library.parameters import SeasonAll

In C:\Users\Jordan Nishimura\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\Jordan Nishimura\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\Jordan Nishimura\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\Jordan Nishimura\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\Jordan Nishimura\Anaconda3\lib\site-

In [2]:
pd.options.display.max_columns=200

## Compiling and Cleaning Data

In [9]:
seasons = ["20{}-{}".format(x, x+1) for x in range(10, 21)]
print(seasons)
def gather_data(seasons = seasons):
    full_player_data = []
    for season in seasons:
        player_gls = pd.read_csv('./data/player_gamelogs_{}.csv'.format(season), dtype={'GAME_ID':'object'})
        player_gls_adv = pd.read_csv('./data/player_advanced_boxscores_{}.csv'.format(season), dtype={'GAME_ID':'object'})
        player_gls_scoring = pd.read_csv('./data/player_scoring_boxscores_{}.csv'.format(season), dtype={'GAME_ID':'object'})
        player_gls_tracking = pd.read_csv('./data/player_tracking_boxscores_{}.csv'.format(season), dtype={'GAME_ID':'object'})
        
        player_gls = player_gls[['SEASON_YEAR', 'PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID',
       'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID', 'GAME_DATE', 'MATCHUP',
       'WL', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM',
       'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK',
       'BLKA', 'PF', 'PFD', 'PTS', 'PLUS_MINUS']]

        player_full = pd.merge(player_gls, player_gls_adv, how='left', on=['GAME_ID', 'PLAYER_ID'], suffixes=['', '_x'])
        player_full = pd.merge(player_full, player_gls_scoring, how='left', on=['GAME_ID', 'PLAYER_ID'], suffixes=['', '_x'])
        player_full = pd.merge(player_full, player_gls_tracking, how='left', on=['GAME_ID', 'PLAYER_ID'], suffixes=['', '_x'])
    
        # drop unnecessary columns
        player_full = player_full.drop(columns = ['PLAYER_ID', 'TEAM_ID', 'TEAM_NAME', 'TEAM_ID_x',
                         'TEAM_ABBREVIATION_x','TEAM_CITY','PLAYER_NAME_x','START_POSITION',
                          'COMMENT','MIN_x','TEAM_ID_x', 'TEAM_ABBREVIATION_x','TEAM_CITY_x', 
                          'PLAYER_NAME_x', 'START_POSITION_x', 'COMMENT_x', 'MIN_x', 
                         'TEAM_ID_x', 'TEAM_ABBREVIATION_x', 'TEAM_CITY_x', 'PLAYER_NAME_x', 
                          'START_POSITION_x', 'COMMENT_x', 'MIN_x', 'AST_x', 'FG_PCT_x',
                         'FG_PCT', 'FG3_PCT', 'FT_PCT', 'E_NET_RATING', 'NET_RATING', 
                         'AST_PCT', 'OREB_PCT', 'DREB_PCT', 'REB_PCT', 'EFG_PCT', 'TS_PCT', 
                         'PCT_FGA_2PT', 'PCT_FGA_3PT', 'PCT_PTS_FT', 'CFG_PCT',
                        'UFG_PCT', 'DFG_PCT'], 
                       errors= 'ignore')

        # Convert date to datetime
        player_full['GAME_DATE'] = pd.to_datetime(player_full['GAME_DATE'])
        
        # Convert W/L to a binary 1/0 for win/loss
        player_full['WL'] = (player_full['WL'].str[0] == 'W').astype(int)
        
        # Add a binary home_game feature
        player_full['HOME_GAME'] = 0 
        player_full['HOME_GAME'] = (player_full['MATCHUP'].str[:6] == player_full['TEAM_ABBREVIATION'].str[:] + ' vs').astype(int)
        
        # convert the percentages into raw numbers (because we will be averaging them, we don't want to average percentages)
        # for example, percentage of points from midrange will be points scored from midrange
        player_full['FG2M'] = player_full['FGM'] - player_full['FG3M']
        player_full['FG2A'] = player_full['FGA'] - player_full['FG3A']
        player_full['PTS_2PT'] = (player_full['PTS'] * player_full['PCT_PTS_2PT']).astype('int8')
        player_full['PTS_2PT_MR'] = (player_full['PTS'] * player_full['PCT_PTS_2PT_MR']).astype('int8')
        player_full['PTS_3PT'] = (player_full['PTS'] * player_full['PCT_PTS_3PT']).astype('int8')
        player_full['PTS_FB'] = (player_full['PTS'] * player_full['PCT_PTS_FB']).astype('int8')
        player_full['PTS_OFF_TOV'] = (player_full['PTS'] * player_full['PCT_PTS_OFF_TOV']).astype('int8')
        player_full['PTS_PAINT'] = (player_full['PTS'] * player_full['PCT_PTS_PAINT']).astype('int8')
        player_full['AST_2PM'] = (player_full['FG2M'] * player_full['PCT_AST_2PM']).astype('int8')
        player_full['UAST_2PM'] = (player_full['FG2M'] * player_full['PCT_UAST_2PM']).astype('int8')
        player_full['AST_3PM'] = (player_full['FG3M'] * player_full['PCT_AST_3PM']).astype('int8')
        player_full['UAST_3PM'] = (player_full['FG3M'] * player_full['PCT_UAST_3PM']).astype('int8')



        player_full = player_full.drop(columns = ['PCT_PTS_2PT', 'PCT_PTS_2PT_MR', 'PCT_PTS_3PT', 
                                                  'PCT_PTS_FB', 'PCT_PTS_OFF_TOV', 'PCT_PTS_PAINT',
                                                  'PCT_AST_2PM', 'PCT_UAST_2PM', 'PCT_AST_3PM',
                                                  'PCT_UAST_3PM', 'PCT_AST_FGM', 'PCT_UAST_FGM', 
                                                  'FGM', 'FGA', 'AST_TOV', 'USG_PCT', 'PACE',
                                                  'PACE_PER40', ], errors='ignore')
        
        
        full_player_data.append(player_full)
        
        
    
    player_df = pd.concat(full_player_data)
    
    
    player_df['home_team_abbr'] = player_df.apply(lambda row: row['TEAM_ABBREVIATION'] if row['HOME_GAME'] == 1 else row['MATCHUP'][-3:], axis=1)
    player_df['away_team_abbr'] = player_df.apply(lambda row: row['TEAM_ABBREVIATION'] if row['HOME_GAME'] == 0 else row['MATCHUP'][-3:], axis=1)
    
    player_df[['home_team_abbr', 'away_team_abbr']] = player_df[['home_team_abbr', 'away_team_abbr']].replace({'NOH':'NOP',
                                                                                                               'NJN':'BKN'})
    
    # Reorder columns
    player_df = player_df[['SEASON_YEAR', 'TEAM_ABBREVIATION', 'PLAYER_NAME', 'home_team_abbr', 
                           'away_team_abbr', 'GAME_ID', 'GAME_DATE', 'MATCHUP',
                           'WL', 'HOME_GAME', 'MIN', 'FG3M', 'FG3A', 'FTM', 
                           'FTA', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL',
                           'BLK', 'BLKA', 'PF', 'PFD', 'PTS', 'PLUS_MINUS', 
                           'E_OFF_RATING', 'OFF_RATING', 'E_DEF_RATING',
                           'DEF_RATING', 'AST_RATIO', 'TM_TOV_PCT', 'E_USG_PCT',
                           'E_PACE', 'POSS', 'PIE', 'SPD', 'DIST', 'ORBC', 
                           'DRBC', 'RBC', 'TCHS', 'SAST', 'FTAST', 'PASS', 
                           'CFGM', 'CFGA', 'UFGM', 'UFGA', 'DFGM', 'DFGA', 
                           'FG2M', 'FG2A', 'PTS_2PT', 'PTS_2PT_MR', 'PTS_3PT',
                           'PTS_FB', 'PTS_OFF_TOV', 'PTS_PAINT', 'AST_2PM',
                           'UAST_2PM', 'AST_3PM', 'UAST_3PM']]
    
    return player_df

player_df = gather_data(seasons)

['2010-11', '2011-12', '2012-13', '2013-14', '2014-15', '2015-16', '2016-17', '2017-18', '2018-19', '2019-20', '2020-21']


In [10]:
player_df['GAME_ID'].nunique()

12207

In [549]:
test = player_df.loc[player_df['GAME_ID'] == '0022000015']
home_test = test.loc[test['HOME_GAME'] == 1].sort_values('MIN', ascending=False)
# home_test.iloc[:12, 9:].values.reshape(1, -1)
home_test
away_test = test.loc[test['HOME_GAME'] == 0].sort_values('MIN', ascending=False)
away_test

,SEASON_YEAR,PLAYER_NAME,home_team_abbr,away_team_abbr,GAME_ID,GAME_DATE,MATCHUP,WL,HOME_GAME,MIN,FG3M,FG3A,FTM,FTA,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,E_OFF_RATING,OFF_RATING,E_DEF_RATING,DEF_RATING,AST_RATIO,TM_TOV_PCT,E_USG_PCT,E_PACE,POSS,PIE,SPD,DIST,ORBC,DRBC,RBC,TCHS,SAST,FTAST,PASS,CFGM,CFGA,UFGM,UFGA,DFGM,DFGA,FG2M,FG2A,PTS_2PT,PTS_2PT_MR,PTS_3PT,PTS_FB,PTS_OFF_TOV,PTS_PAINT,AST_2PM,UAST_2PM,AST_3PM,UAST_3PM
6676,2020-21,Solomon Hill,CHI,ATL,0022000015,2020-12-23,ATL @ CHI,1,0,28.400000,0,3,0,0,0,3,3,0,3,0,1,0,2,0,0,-8,90.9,90.3,100.3,101.6,0.0,50.0,0.089,105.97,62,-0.051,4.16,2.11,1,8,9,41,0,0,33,0,0,0,3,2,3,0,0,0,0,0,0,0,0,0,0,0,0
6782,2020-21,Kevin Huerter,CHI,ATL,0022000015,2020-12-23,ATL @ CHI,1,0,27.050000,2,6,1,2,0,2,2,4,0,1,0,0,2,1,11,0,108.6,106.6,104.2,103.2,26.7,0.0,0.172,108.42,61,0.082,4.27,2.06,1,2,3,47,2,0,36,1,2,3,8,0,0,2,4,4,2,5,5,0,2,1,1,2,0
6791,2020-21,Trae Young,CHI,ATL,0022000015,2020-12-23,ATL @ CHI,1,0,25.963333,5,6,12,14,2,4,6,7,4,0,0,0,1,9,37,30,143.4,142.4,93.4,96.4,25.0,14.3,0.336,107.60,59,0.362,4.22,1.96,2,5,7,71,0,0,45,4,4,6,8,1,2,5,6,9,1,14,1,7,7,0,5,0,5
6617,2020-21,De'Andre Hunter,CHI,ATL,0022000015,2020-12-23,ATL @ CHI,1,0,25.900000,1,1,0,0,0,3,3,1,2,0,0,0,3,0,11,40,136.7,141.0,78.0,76.7,11.1,22.2,0.121,112.94,61,0.082,4.24,1.94,1,4,5,25,1,0,16,4,4,1,2,2,2,4,5,7,0,3,0,4,7,3,1,1,0
6806,2020-21,Bogdan Bogdanovic,CHI,ATL,0022000015,2020-12-23,ATL @ CHI,1,0,24.933333,3,8,2,3,0,7,7,2,2,1,0,1,2,1,15,0,105.0,105.1,102.7,101.6,11.8,11.8,0.246,114.93,59,0.131,4.40,1.96,1,11,12,50,1,0,36,0,1,5,11,1,2,2,4,4,0,9,4,4,4,1,1,3,0
6802,2020-21,Danilo Gallinari,CHI,ATL,0022000015,2020-12-23,ATL @ CHI,1,0,24.433333,1,2,4,4,0,4,4,3,3,0,0,0,2,4,13,31,144.4,149.1,89.5,89.5,21.4,21.4,0.187,111.74,55,0.119,3.91,1.72,0,5,5,42,0,0,32,2,2,2,4,2,3,3,4,6,2,3,0,0,4,2,0,1,0
6817,2020-21,Cam Reddish,CHI,ATL,0022000015,2020-12-23,ATL @ CHI,1,0,23.766667,1,3,2,2,1,5,6,0,1,2,0,0,3,1,15,27,137.2,139.6,87.1,87.0,0.0,7.7,0.228,108.94,53,0.138,4.52,1.91,1,7,8,30,0,0,15,3,5,3,6,2,2,5,8,10,0,3,4,6,10,2,3,1,0
6686,2020-21,Bruno Fernando,CHI,ATL,0022000015,2020-12-23,ATL @ CHI,1,0,18.133333,0,0,1,2,3,4,7,1,3,2,0,0,3,1,5,-1,80.1,80.0,85.5,84.6,12.5,37.5,0.156,103.98,40,0.133,4.18,1.35,6,6,11,31,0,0,25,1,2,1,1,1,4,2,3,4,0,0,2,4,4,2,0,0,0
6840,2020-21,John Collins,CHI,ATL,0022000015,2020-12-23,ATL @ CHI,1,0,18.050000,0,2,2,2,0,4,4,2,0,0,2,1,4,5,14,21,144.0,147.5,93.8,90.5,16.7,0.0,0.227,108.34,40,0.165,4.28,1.38,1,8,9,30,0,0,19,5,7,1,3,2,5,6,8,11,2,0,4,2,9,4,1,0,0
6703,2020-21,Brandon Goodwin,CHI,ATL,0022000015,2020-12-23,ATL @ CHI,1,0,13.236667,1,3,0,0,0,1,1,4,1,0,0,1,4,0,3,-16,58.8,59.3,111.4,110.3,44.4,11.1,0.160,101.39,27,0.000,4.02,0.96,1,1,2,40,0,0,32,0,1,1,3,1,1,0,1,0,0,3,0,0,0,0,0,0,1


In [593]:
test = player_df.loc[player_df['SEASON_YEAR'] == '2018-19']

home = test.loc[test['HOME_GAME'] == 1]
home.loc[home['GAME_ID'] == '0021801228']

away = test.loc[test['HOME_GAME'] == 0]

away.loc[away['GAME_ID'] == '0021801228']

,SEASON_YEAR,PLAYER_NAME,home_team_abbr,away_team_abbr,GAME_ID,GAME_DATE,MATCHUP,WL,HOME_GAME,MIN,FG3M,FG3A,FTM,FTA,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,E_OFF_RATING,OFF_RATING,E_DEF_RATING,DEF_RATING,AST_RATIO,TM_TOV_PCT,E_USG_PCT,E_PACE,POSS,PIE,SPD,DIST,ORBC,DRBC,RBC,TCHS,SAST,FTAST,PASS,CFGM,CFGA,UFGM,UFGA,DFGM,DFGA,FG2M,FG2A,PTS_2PT,PTS_2PT_MR,PTS_3PT,PTS_FB,PTS_OFF_TOV,PTS_PAINT,AST_2PM,UAST_2PM,AST_3PM,UAST_3PM
3,2018-19,Dario Saric,DEN,MIN,0021801228,2019-04-10,MIN @ DEN,0,0,29.255000,0,5,3,3,1,6,7,3,1,0,0,1,3,2,11,-1,93.8,93.2,101.3,94.9,16.7,5.6,0.236,93.46,59,0.081,4.03,2.12,5,12,17,62,0,0,44,3,4,1,9,2,6,4,8,7,2,0,0,0,5,2,2,0,0
51,2018-19,Tyus Jones,DEN,MIN,0021801228,2019-04-10,MIN @ DEN,0,0,34.266667,0,2,0,0,0,1,1,8,0,0,0,0,4,0,8,-19,84.3,83.1,110.5,108.3,40.0,0.0,0.160,98.47,71,0.040,4.29,2.61,0,4,4,91,1,0,74,3,9,1,3,2,2,4,10,8,2,0,4,4,6,2,2,0,0
65,2018-19,Andrew Wiggins,DEN,MIN,0021801228,2019-04-10,MIN @ DEN,0,0,35.983333,3,6,0,1,1,4,5,4,2,0,0,0,1,3,25,-8,89.9,90.4,103.2,98.7,15.4,7.7,0.277,96.82,73,0.167,3.80,2.45,4,6,9,56,2,0,31,4,6,7,14,1,4,8,14,16,4,9,3,0,12,1,7,3,0
81,2018-19,Gorgui Dieng,DEN,MIN,0021801228,2019-04-10,MIN @ DEN,0,0,34.445000,2,4,0,0,4,7,11,2,5,3,0,1,5,2,18,-19,88.9,87.7,114.3,112.2,7.4,18.5,0.321,100.75,73,0.072,4.07,2.50,5,12,17,70,0,0,46,4,8,4,12,4,4,6,16,12,3,5,0,0,7,3,3,2,0
93,2018-19,Josh Okogie,DEN,MIN,0021801228,2019-04-10,MIN @ DEN,0,0,17.116667,0,0,0,0,0,1,1,1,0,1,0,1,1,1,0,-3,86.1,83.8,97.8,94.4,25.0,0.0,0.073,99.22,37,-0.017,4.28,1.30,1,3,4,15,0,0,11,0,2,0,1,0,1,0,3,0,0,0,0,0,0,0,0,0,0
96,2018-19,Jared Terrell,DEN,MIN,0021801228,2019-04-10,MIN @ DEN,0,0,5.716667,0,1,1,2,0,0,0,0,0,0,0,0,1,1,1,2,127.5,118.2,111.3,100.0,0.0,0.0,0.179,84.30,11,-0.098,4.35,0.45,0,0,0,11,0,0,9,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
112,2018-19,C.J. Williams,DEN,MIN,0021801228,2019-04-10,MIN @ DEN,0,0,17.600000,2,2,0,0,0,1,1,1,0,0,0,0,2,0,6,11,135.6,137.5,96.2,97.1,33.3,0.0,0.060,91.04,32,0.078,4.15,1.30,0,2,2,16,1,0,14,0,0,2,2,2,3,0,0,0,0,6,0,0,0,0,0,2,0
153,2018-19,Anthony Tolliver,DEN,MIN,0021801228,2019-04-10,MIN @ DEN,0,0,21.083333,1,5,0,0,0,2,2,3,1,1,0,0,2,1,3,5,110.0,111.6,99.3,100.0,33.3,11.1,0.129,98.99,43,0.024,4.22,1.59,0,4,4,31,0,0,24,0,1,1,4,2,4,0,0,0,0,3,0,0,0,0,0,1,0
203,2018-19,Cameron Reynolds,DEN,MIN,0021801228,2019-04-10,MIN @ DEN,0,0,25.950000,5,7,0,0,0,6,6,1,0,0,0,0,3,2,19,6,114.0,117.6,109.6,105.9,8.3,0.0,0.201,94.26,51,0.188,4.28,2.01,2,6,7,32,0,0,21,2,2,5,9,0,0,2,4,4,1,14,0,6,1,1,1,5,0
220,2018-19,Keita Bates-Diop,DEN,MIN,0021801228,2019-04-10,MIN @ DEN,0,0,18.583333,0,0,0,0,1,6,7,1,1,1,0,1,0,0,4,6,102.3,100.0,81.4,82.9,16.7,16.7,0.142,90.20,35,0.179,4.05,1.35,1,8,9,26,0,0,20,2,4,0,0,0,0,2,4,4,0,0,0,2,4,2,0,0,0


In [17]:
test = player_df.loc[player_df['GAME_ID'] == '0021801220']
home_team = test.loc[test['HOME_GAME'] == 1].sort_values('MIN', ascending=False)

home_matchup_info = home_team[['SEASON_YEAR', 'TEAM_ABBREVIATION',
                               'home_team_abbr', 'away_team_abbr',
                               'GAME_ID', 'GAME_DATE', 'MATCHUP',
                               'WL', 'HOME_GAME']].iloc[0].values.reshape(1, -1)

home_matchup_info
home_team.iloc[:12, 10:]

,MIN,FG3M,FG3A,FTM,FTA,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,E_OFF_RATING,OFF_RATING,E_DEF_RATING,DEF_RATING,AST_RATIO,TM_TOV_PCT,E_USG_PCT,E_PACE,POSS,PIE,SPD,DIST,ORBC,DRBC,RBC,TCHS,SAST,FTAST,PASS,CFGM,CFGA,UFGM,UFGA,DFGM,DFGA,FG2M,FG2A,PTS_2PT,PTS_2PT_MR,PTS_3PT,PTS_FB,PTS_OFF_TOV,PTS_PAINT,AST_2PM,UAST_2PM,AST_3PM,UAST_3PM
26,35.816667,1,2,3,4,9,16,25,6,3,0,1,1,2,5,20,9,118.6,122.9,111.6,116.3,23.1,11.5,0.194,113.48,83,0.193,4.23,2.75,14,19,32,88,0,0,65,7,12,1,3,3,6,7,13,14,0,3,2,2,14,4,2,1,0
14,33.700000,1,7,10,11,2,3,5,11,6,2,0,2,0,10,23,18,128.6,133.3,111.6,111.7,28.2,15.4,0.292,112.47,78,0.120,4.30,2.63,7,6,12,95,1,0,62,4,6,2,12,1,1,5,11,10,0,2,6,4,10,1,4,0,1
205,33.616667,5,8,2,3,0,5,5,2,1,0,1,1,2,2,23,9,124.2,129.3,112.5,115.8,9.5,4.8,0.206,111.63,75,0.103,3.91,2.39,1,6,7,41,0,0,19,2,5,6,12,2,3,3,9,6,0,14,0,0,6,0,2,5,0
6,30.866667,2,6,6,8,6,4,10,3,0,0,3,1,3,4,20,13,124.1,129.6,112.3,108.2,12.5,0.0,0.236,112.34,71,0.100,4.08,2.30,9,10,19,54,1,0,33,0,4,6,13,3,7,4,11,8,0,6,0,0,8,4,0,2,0
213,30.366667,5,6,0,0,1,4,5,3,3,1,1,0,4,0,17,6,123.7,129.0,118.0,118.6,21.4,21.4,0.128,112.45,69,0.109,4.20,2.33,1,6,7,31,1,0,21,1,2,5,6,5,6,1,2,2,0,14,0,2,2,0,1,4,1
218,20.116667,0,1,3,3,1,1,2,1,0,1,0,0,5,2,11,-11,107.0,110.9,133.2,134.8,10.0,0.0,0.162,112.43,46,0.068,4.28,1.59,3,1,4,27,0,0,18,2,4,2,4,3,4,4,7,7,0,0,2,0,7,4,0,0,0
169,17.066667,2,6,1,2,0,2,2,2,1,0,0,1,2,1,7,-16,101.3,105.0,153.4,148.7,14.3,7.1,0.243,111.49,40,-0.014,4.16,1.33,1,3,4,26,0,1,14,0,5,2,5,2,4,0,4,0,0,5,0,0,0,0,0,1,1
145,12.870000,1,1,2,2,1,2,3,0,2,1,0,0,2,1,7,-8,116.3,110.3,126.6,125.0,0.0,33.3,0.179,110.25,29,0.095,4.09,0.96,1,2,3,16,0,0,9,0,0,2,3,1,1,1,2,2,0,3,2,0,2,0,1,1,0
101,12.066667,0,3,1,2,0,0,0,0,0,0,1,0,2,2,1,-13,90.5,96.4,149.3,142.9,0.0,0.0,0.136,112.65,28,-0.136,3.87,0.88,0,0,0,28,0,0,23,0,0,0,4,2,3,0,1,0,0,0,0,0,0,0,0,0,0
103,9.016667,0,1,1,1,0,2,2,1,1,0,0,1,2,0,1,-6,91.5,105.0,147.7,135.0,25.0,25.0,0.111,109.77,20,-0.033,4.31,0.73,1,2,3,22,1,0,19,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [20]:
def long_to_wide(df = player_df):
    wide_data = np.empty((0, 657))
    
    # All game_ids
    game_ids = df['GAME_ID'].unique()
    
    # 
    new_cols = ['SEASON_YEAR', 'TEAM_ABBREVIATION', 'home_team_abbr', 
               'away_team_abbr', 'GAME_ID', 'GAME_DATE', 'MATCHUP',
               'WL', 'HOME_GAME']
    
    for game_id in tqdm(game_ids, desc="Progress"):
        
        # Select all the players boxscores from one game
        game_df = df.loc[df['GAME_ID'] == game_id]
               
        # Select the home team players, sort by minutes
        home_team = game_df.loc[game_df['HOME_GAME'] == 1].sort_values('MIN', ascending=False)
        
        # Select matchup info
        home_matchup_info = home_team[['SEASON_YEAR', 'TEAM_ABBREVIATION',
                                       'home_team_abbr', 'away_team_abbr',
                                       'GAME_ID', 'GAME_DATE', 'MATCHUP',
                                       'WL', 'HOME_GAME']].iloc[0].values.reshape(1, -1)
        
        # unroll the home players' stats up to player 12 into one long row
        home_stats_row = home_team.iloc[:12, 10:].values.reshape(1, -1)
        
        # if 
        if home_stats_row.shape[1] < 648:
            missing_stats = 648 - home_stats_row.shape[1]
            padding = np.zeros((1, missing_stats))
            home_stats_row = np.append(home_stats_row, padding, axis=1)      
        
        home_stats_row = np.append(home_matchup_info, home_stats_row, axis=1)
        
    
        away_team = game_df.loc[game_df['HOME_GAME'] == 0].sort_values('MIN', ascending=False)
        
        away_matchup_info = away_team[['SEASON_YEAR', 'TEAM_ABBREVIATION',
                                       'home_team_abbr', 'away_team_abbr',
                                       'GAME_ID', 'GAME_DATE', 'MATCHUP',
                                       'WL', 'HOME_GAME']].iloc[0].values.reshape(1, -1)
        
        away_stats_row = away_team.iloc[:12, 10:].values.reshape(1, -1)

        if away_stats_row.shape[1] < 648:
            missing_stats = 648 - away_stats_row.shape[1]
            padding = np.zeros((1, missing_stats))
            away_stats_row = np.append(away_stats_row, padding, axis=1)        
        
        away_stats_row = np.append(away_matchup_info, away_stats_row, axis=1)
                
        wide_data = np.append(wide_data, home_stats_row, axis=0)
        wide_data = np.append(wide_data, away_stats_row, axis=0)
        

    for i in range(1, 13):
        player_cols = ['P{}_'.format(i) + x for x in df.columns[10:]]
        new_cols.extend(player_cols)
        
    wide_df = pd.DataFrame(wide_data, columns = new_cols)
    
    return wide_df


wide_df = long_to_wide(df=player_df)
wide_df.to_csv("player_boxscores_wideform.csv", index=False)
wide_df
    

Progress: 100%|██████████████████████████████████████████████████████████████████| 12207/12207 [46:17<00:00,  4.40it/s]


,SEASON_YEAR,TEAM_ABBREVIATION,home_team_abbr,away_team_abbr,GAME_ID,GAME_DATE,MATCHUP,WL,HOME_GAME,P1_MIN,P1_FG3M,P1_FG3A,P1_FTM,P1_FTA,P1_OREB,P1_DREB,P1_REB,P1_AST,P1_TOV,P1_STL,P1_BLK,P1_BLKA,P1_PF,P1_PFD,P1_PTS,P1_PLUS_MINUS,P1_E_OFF_RATING,P1_OFF_RATING,P1_E_DEF_RATING,P1_DEF_RATING,P1_AST_RATIO,P1_TM_TOV_PCT,P1_E_USG_PCT,P1_E_PACE,P1_POSS,P1_PIE,P1_SPD,P1_DIST,P1_ORBC,P1_DRBC,P1_RBC,P1_TCHS,P1_SAST,P1_FTAST,P1_PASS,P1_CFGM,P1_CFGA,P1_UFGM,P1_UFGA,P1_DFGM,P1_DFGA,P1_FG2M,P1_FG2A,P1_PTS_2PT,P1_PTS_2PT_MR,P1_PTS_3PT,P1_PTS_FB,P1_PTS_OFF_TOV,P1_PTS_PAINT,P1_AST_2PM,P1_UAST_2PM,P1_AST_3PM,P1_UAST_3PM,P2_MIN,P2_FG3M,P2_FG3A,P2_FTM,P2_FTA,P2_OREB,P2_DREB,P2_REB,P2_AST,P2_TOV,P2_STL,P2_BLK,P2_BLKA,P2_PF,P2_PFD,P2_PTS,P2_PLUS_MINUS,P2_E_OFF_RATING,P2_OFF_RATING,P2_E_DEF_RATING,P2_DEF_RATING,P2_AST_RATIO,P2_TM_TOV_PCT,P2_E_USG_PCT,P2_E_PACE,P2_POSS,P2_PIE,P2_SPD,P2_DIST,P2_ORBC,P2_DRBC,P2_RBC,P2_TCHS,P2_SAST,P2_FTAST,P2_PASS,P2_CFGM,...,P11_AST,P11_TOV,P11_STL,P11_BLK,P11_BLKA,P11_PF,P11_PFD,P11_PTS,P11_PLUS_MINUS,P11_E_OFF_RATING,P11_OFF_RATING,P11_E_DEF_RATING,P11_DEF_RATING,P11_AST_RATIO,P11_TM_TOV_PCT,P11_E_USG_PCT,P11_E_PACE,P11_POSS,P11_PIE,P11_SPD,P11_DIST,P11_ORBC,P11_DRBC,P11_RBC,P11_TCHS,P11_SAST,P11_FTAST,P11_PASS,P11_CFGM,P11_CFGA,P11_UFGM,P11_UFGA,P11_DFGM,P11_DFGA,P11_FG2M,P11_FG2A,P11_PTS_2PT,P11_PTS_2PT_MR,P11_PTS_3PT,P11_PTS_FB,P11_PTS_OFF_TOV,P11_PTS_PAINT,P11_AST_2PM,P11_UAST_2PM,P11_AST_3PM,P11_UAST_3PM,P12_MIN,P12_FG3M,P12_FG3A,P12_FTM,P12_FTA,P12_OREB,P12_DREB,P12_REB,P12_AST,P12_TOV,P12_STL,P12_BLK,P12_BLKA,P12_PF,P12_PFD,P12_PTS,P12_PLUS_MINUS,P12_E_OFF_RATING,P12_OFF_RATING,P12_E_DEF_RATING,P12_DEF_RATING,P12_AST_RATIO,P12_TM_TOV_PCT,P12_E_USG_PCT,P12_E_PACE,P12_POSS,P12_PIE,P12_SPD,P12_DIST,P12_ORBC,P12_DRBC,P12_RBC,P12_TCHS,P12_SAST,P12_FTAST,P12_PASS,P12_CFGM,P12_CFGA,P12_UFGM,P12_UFGA,P12_DFGM,P12_DFGA,P12_FG2M,P12_FG2A,P12_PTS_2PT,P12_PTS_2PT_MR,P12_PTS_3PT,P12_PTS_FB,P12_PTS_OFF_TOV,P12_PTS_PAINT,P12_AST_2PM,P12_UAST_2PM,P12_AST_3PM,P12_UAST_3PM
0,2010-11,OKC,OKC,MIL,0021001225,2011-04-13,OKC vs. MIL,0,1,31.8333,0,0,2,4,3,7,10,0,4,2,1,2,2,3,12,-14,95.6,96.8,120.5,119,0,23.5,0.23,95.06,63,0.078,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,11,9,2,0,2,2,8,4,1,0,0,29.1783,0,0,2,4,1,4,5,0,2,1,0,1,2,2,10,-22,90.1,90,130.8,128.8,0,15.4,0.191,97.12,60,0.041,0,0,0,0,0,0,0,0,0,0,...,3,2,1,0,0,1,1,2,-8,86.9,84.6,120.2,120,37.5,25,0.183,99.09,26,0.047,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,2,0,0,0,0,2,0,1,0,0,8.2,1,2,0,0,0,1,1,0,0,0,0,0,2,0,3,-13,70.6,75,152.4,147.1,0,0,0.111,97.76,16,0.033,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,1,0
1,2010-11,MIL,OKC,MIL,0021001225,2011-04-13,MIL @ OKC,1,0,36.1883,1,3,7,8,0,6,6,7,3,5,1,2,4,6,16,6,98.7,97.3,89.8,87.8,28,12,0.234,95.71,73,0.159,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,8,6,0,3,4,5,6,0,3,0,1,33.9667,1,2,3,3,2,7,9,0,0,2,2,0,2,4,14,12,120.8,119.4,99.6,101.5,0,0,0.125,95.05,67,0.156,0,0,0,0,0,0,0,0,0,0,...,3,1,0,0,0,1,1,6,2,144.1,133.3,111.9,120,37.5,12.5,0.36,94.61,15,0.141,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,6,3,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2010-11,DAL,DAL,NOP,0021001223,2011-04-13,DAL vs. NOH,1,1,31.785,1,3,11,12,1,5,6,2,3,0,0,0,2,9,32,21,121.3,124.6,95.3,93.8,7.1,10.7,0.372,97.98,65,0.17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,18,18,6,3,0,4,12,3,5,1,0,29.0983,0,1,2,4,0,8,8,4,3,1,0,0,1,2,10,11,120.3,121.1,99,101.8,22.2,16.7,0.228,95.64,57,0.108,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,3,0,2,-2,123,133.3,143.4,155.6,20,0,0.34,99.32,9,-0.158,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,3,2,2,0,2,0,0,1,0,0,0,3.41667,0,0,0,0,1,0,1,0,1,0,0,0,0,0,2,8,173.6,142.9,33.3,33.3,0,50,0.296,82.61,7,0.15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,2,2,0,0,0,0,1,0,0,0
3,2010-11,NOH,DAL,NOP,0021001223,2011-04-13,NOH @ DAL,0,0,28.6583,1,1,5,6,0,0,0,7,1,0,1,0,2,4,22,-22,100.3,101.8,137,136.2,28,4,0.296,95.91,56,0.144,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,14,13,6,2,2,0,8,2,4,1,0,26.015,0,0,2,2,1,5,6,1,3,1,0,0,3,2,6,-15,88.7,90.2,118.2,119.6,8.3,25,0.

In [3]:
wide_df = pd.read_csv("player_boxscores_wideform.csv")
wide_df

,SEASON_YEAR,TEAM_ABBREVIATION,home_team_abbr,away_team_abbr,GAME_ID,GAME_DATE,MATCHUP,WL,HOME_GAME,P1_MIN,P1_FG3M,P1_FG3A,P1_FTM,P1_FTA,P1_OREB,P1_DREB,P1_REB,P1_AST,P1_TOV,P1_STL,P1_BLK,P1_BLKA,P1_PF,P1_PFD,P1_PTS,P1_PLUS_MINUS,P1_E_OFF_RATING,P1_OFF_RATING,P1_E_DEF_RATING,P1_DEF_RATING,P1_AST_RATIO,P1_TM_TOV_PCT,P1_E_USG_PCT,P1_E_PACE,P1_POSS,P1_PIE,P1_SPD,P1_DIST,P1_ORBC,P1_DRBC,P1_RBC,P1_TCHS,P1_SAST,P1_FTAST,P1_PASS,P1_CFGM,P1_CFGA,P1_UFGM,P1_UFGA,P1_DFGM,P1_DFGA,P1_FG2M,P1_FG2A,P1_PTS_2PT,P1_PTS_2PT_MR,P1_PTS_3PT,P1_PTS_FB,P1_PTS_OFF_TOV,P1_PTS_PAINT,P1_AST_2PM,P1_UAST_2PM,P1_AST_3PM,P1_UAST_3PM,P2_MIN,P2_FG3M,P2_FG3A,P2_FTM,P2_FTA,P2_OREB,P2_DREB,P2_REB,P2_AST,P2_TOV,P2_STL,P2_BLK,P2_BLKA,P2_PF,P2_PFD,P2_PTS,P2_PLUS_MINUS,P2_E_OFF_RATING,P2_OFF_RATING,P2_E_DEF_RATING,P2_DEF_RATING,P2_AST_RATIO,P2_TM_TOV_PCT,P2_E_USG_PCT,P2_E_PACE,P2_POSS,P2_PIE,P2_SPD,P2_DIST,P2_ORBC,P2_DRBC,P2_RBC,P2_TCHS,P2_SAST,P2_FTAST,P2_PASS,P2_CFGM,...,P11_AST,P11_TOV,P11_STL,P11_BLK,P11_BLKA,P11_PF,P11_PFD,P11_PTS,P11_PLUS_MINUS,P11_E_OFF_RATING,P11_OFF_RATING,P11_E_DEF_RATING,P11_DEF_RATING,P11_AST_RATIO,P11_TM_TOV_PCT,P11_E_USG_PCT,P11_E_PACE,P11_POSS,P11_PIE,P11_SPD,P11_DIST,P11_ORBC,P11_DRBC,P11_RBC,P11_TCHS,P11_SAST,P11_FTAST,P11_PASS,P11_CFGM,P11_CFGA,P11_UFGM,P11_UFGA,P11_DFGM,P11_DFGA,P11_FG2M,P11_FG2A,P11_PTS_2PT,P11_PTS_2PT_MR,P11_PTS_3PT,P11_PTS_FB,P11_PTS_OFF_TOV,P11_PTS_PAINT,P11_AST_2PM,P11_UAST_2PM,P11_AST_3PM,P11_UAST_3PM,P12_MIN,P12_FG3M,P12_FG3A,P12_FTM,P12_FTA,P12_OREB,P12_DREB,P12_REB,P12_AST,P12_TOV,P12_STL,P12_BLK,P12_BLKA,P12_PF,P12_PFD,P12_PTS,P12_PLUS_MINUS,P12_E_OFF_RATING,P12_OFF_RATING,P12_E_DEF_RATING,P12_DEF_RATING,P12_AST_RATIO,P12_TM_TOV_PCT,P12_E_USG_PCT,P12_E_PACE,P12_POSS,P12_PIE,P12_SPD,P12_DIST,P12_ORBC,P12_DRBC,P12_RBC,P12_TCHS,P12_SAST,P12_FTAST,P12_PASS,P12_CFGM,P12_CFGA,P12_UFGM,P12_UFGA,P12_DFGM,P12_DFGA,P12_FG2M,P12_FG2A,P12_PTS_2PT,P12_PTS_2PT_MR,P12_PTS_3PT,P12_PTS_FB,P12_PTS_OFF_TOV,P12_PTS_PAINT,P12_AST_2PM,P12_UAST_2PM,P12_AST_3PM,P12_UAST_3PM
0,2010-11,OKC,OKC,MIL,21001225,2011-04-13,OKC vs. MIL,0,1,31.833333,0.0,0.0,2.0,4.0,3.0,7.0,10.0,0.0,4.0,2.0,1.0,2.0,2.0,3.0,12.0,-14.0,95.6,96.8,120.5,119.0,0.0,23.5,0.230,95.06,63.0,0.078,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,11.0,9.0,2.0,0.0,2.0,2.0,8.0,4.0,1.0,0.0,0.0,29.178333,0.0,0.0,2.0,4.0,1.0,4.0,5.0,0.0,2.0,1.0,0.0,1.0,2.0,2.0,10.0,-22.0,90.1,90.0,130.8,128.8,0.0,15.4,0.191,97.12,60.0,0.041,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,2.0,1.0,0.0,0.0,1.0,1.0,2.0,-8.0,86.9,84.6,120.2,120.0,37.5,25.0,0.183,99.09,26.0,0.047,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,8.200000,1.0,2.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3.0,-13.0,70.6,75.0,152.4,147.1,0.0,0.0,0.111,97.76,16.0,0.033,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,2010-11,MIL,OKC,MIL,21001225,2011-04-13,MIL @ OKC,1,0,36.188333,1.0,3.0,7.0,8.0,0.0,6.0,6.0,7.0,3.0,5.0,1.0,2.0,4.0,6.0,16.0,6.0,98.7,97.3,89.8,87.8,28.0,12.0,0.234,95.71,73.0,0.159,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,8.0,6.0,0.0,3.0,4.0,5.0,6.0,0.0,3.0,0.0,1.0,33.966667,1.0,2.0,3.0,3.0,2.0,7.0,9.0,0.0,0.0,2.0,2.0,0.0,2.0,4.0,14.0,12.0,120.8,119.4,99.6,101.5,0.0,0.0,0.125,95.05,67.0,0.156,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,1.0,0.0,0.0,0.0,1.0,1.0,6.0,2.0,144.1,133.3,111.9,120.0,37.5,12.5,0.360,94.61,15.0,0.141,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,6.0,3.0,0.0,0.0,0.0,0.0,2.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.00,0.0,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2010-11,DAL,DAL,NOP,21001223,2011-04-13,DAL vs. NOH,1,1,31.785000,1.0,3.0,11.0,12.0,1.0,5.0,6.0,2.0,3.0,0.0,0.0,0.0,2.0,9.0,32.0,21.0,121.3,124.6,95.3,93.8,7.1,10.7,0.372,

In [22]:
test = wide_df.loc[wide_df['SEASON_YEAR'] == '2018-19']
test = test.loc[test['TEAM_ABBREVIATION'] == 'ATL'].sort_values('GAME_DATE')

test

,SEASON_YEAR,TEAM_ABBREVIATION,home_team_abbr,away_team_abbr,GAME_ID,GAME_DATE,MATCHUP,WL,HOME_GAME,P1_MIN,P1_FG3M,P1_FG3A,P1_FTM,P1_FTA,P1_OREB,P1_DREB,P1_REB,P1_AST,P1_TOV,P1_STL,P1_BLK,P1_BLKA,P1_PF,P1_PFD,P1_PTS,P1_PLUS_MINUS,P1_E_OFF_RATING,P1_OFF_RATING,P1_E_DEF_RATING,P1_DEF_RATING,P1_AST_RATIO,P1_TM_TOV_PCT,P1_E_USG_PCT,P1_E_PACE,P1_POSS,P1_PIE,P1_SPD,P1_DIST,P1_ORBC,P1_DRBC,P1_RBC,P1_TCHS,P1_SAST,P1_FTAST,P1_PASS,P1_CFGM,P1_CFGA,P1_UFGM,P1_UFGA,P1_DFGM,P1_DFGA,P1_FG2M,P1_FG2A,P1_PTS_2PT,P1_PTS_2PT_MR,P1_PTS_3PT,P1_PTS_FB,P1_PTS_OFF_TOV,P1_PTS_PAINT,P1_AST_2PM,P1_UAST_2PM,P1_AST_3PM,P1_UAST_3PM,P2_MIN,P2_FG3M,P2_FG3A,P2_FTM,P2_FTA,P2_OREB,P2_DREB,P2_REB,P2_AST,P2_TOV,P2_STL,P2_BLK,P2_BLKA,P2_PF,P2_PFD,P2_PTS,P2_PLUS_MINUS,P2_E_OFF_RATING,P2_OFF_RATING,P2_E_DEF_RATING,P2_DEF_RATING,P2_AST_RATIO,P2_TM_TOV_PCT,P2_E_USG_PCT,P2_E_PACE,P2_POSS,P2_PIE,P2_SPD,P2_DIST,P2_ORBC,P2_DRBC,P2_RBC,P2_TCHS,P2_SAST,P2_FTAST,P2_PASS,P2_CFGM,...,P11_AST,P11_TOV,P11_STL,P11_BLK,P11_BLKA,P11_PF,P11_PFD,P11_PTS,P11_PLUS_MINUS,P11_E_OFF_RATING,P11_OFF_RATING,P11_E_DEF_RATING,P11_DEF_RATING,P11_AST_RATIO,P11_TM_TOV_PCT,P11_E_USG_PCT,P11_E_PACE,P11_POSS,P11_PIE,P11_SPD,P11_DIST,P11_ORBC,P11_DRBC,P11_RBC,P11_TCHS,P11_SAST,P11_FTAST,P11_PASS,P11_CFGM,P11_CFGA,P11_UFGM,P11_UFGA,P11_DFGM,P11_DFGA,P11_FG2M,P11_FG2A,P11_PTS_2PT,P11_PTS_2PT_MR,P11_PTS_3PT,P11_PTS_FB,P11_PTS_OFF_TOV,P11_PTS_PAINT,P11_AST_2PM,P11_UAST_2PM,P11_AST_3PM,P11_UAST_3PM,P12_MIN,P12_FG3M,P12_FG3A,P12_FTM,P12_FTA,P12_OREB,P12_DREB,P12_REB,P12_AST,P12_TOV,P12_STL,P12_BLK,P12_BLKA,P12_PF,P12_PFD,P12_PTS,P12_PLUS_MINUS,P12_E_OFF_RATING,P12_OFF_RATING,P12_E_DEF_RATING,P12_DEF_RATING,P12_AST_RATIO,P12_TM_TOV_PCT,P12_E_USG_PCT,P12_E_PACE,P12_POSS,P12_PIE,P12_SPD,P12_DIST,P12_ORBC,P12_DRBC,P12_RBC,P12_TCHS,P12_SAST,P12_FTAST,P12_PASS,P12_CFGM,P12_CFGA,P12_UFGM,P12_UFGA,P12_DFGM,P12_DFGA,P12_FG2M,P12_FG2A,P12_PTS_2PT,P12_PTS_2PT_MR,P12_PTS_3PT,P12_PTS_FB,P12_PTS_OFF_TOV,P12_PTS_PAINT,P12_AST_2PM,P12_UAST_2PM,P12_AST_3PM,P12_UAST_3PM
21643,2018-19,ATL,NYK,ATL,0021800007,2018-10-17,ATL @ NYK,0,0,33.1333,1,5,3,4,0,6,6,5,4,0,0,2,4,6,14,-20,99.2,98.8,116.3,126.9,20,16,0.236,119.31,80,0.053,4.52,2.76,2,8,10,74,0,1,48,3,7,2,7,1,1,4,9,7,2,2,0,0,6,0,4,0,1,32.3167,3,7,4,4,0,6,6,6,6,0,0,0,4,4,21,-23,104.4,103.8,128.5,133.3,20.7,20.7,0.282,117.73,78,0.093,4.4,2.56,1,10,10,53,0,0,28,2,...,0,0,1,1,0,1,0,7,2,105.4,110,103.5,105.3,0,0,0.168,116.01,20,0.316,4.44,0.69,4,3,6,21,0,0,16,1,1,2,3,1,3,2,2,3,0,3,0,2,3,1,1,1,0,4.58333,0,0,0,0,1,2,3,0,0,0,0,1,0,0,0,4,76.6,80,40.5,36.4,0,0,0.08,106.4,10,0.429,3.97,0.33,2,2,4,6,0,0,5,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
21615,2018-19,ATL,MEM,ATL,0021800020,2018-10-19,ATL @ MEM,0,0,33.0833,4,9,2,3,1,1,2,9,3,1,1,1,2,3,20,-11,112.3,112.2,121.8,127,29,9.7,0.265,109.63,74,0.104,4.52,2.71,3,3,6,93,1,0,66,1,4,6,14,1,1,3,9,6,0,12,13,3,6,2,0,3,1,32.4333,1,2,3,6,0,3,3,1,5,1,1,0,4,3,12,-8,111.5,109.7,123.3,122.5,5,25,0.238,104.66,72,-0.011,4.43,2.6,2,5,6,47,1,0,29,2,...,2,0,0,0,1,1,0,3,-6,132.2,132,149.5,144.4,40,0,0.1,101.38,25,0.03,4.25,0.94,0,0,0,12,0,0,9,0,1,1,2,1,1,0,2,0,0,3,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
21583,2018-19,ATL,CLE,ATL,0021800036,2018-10-21,ATL @ CLE,1,0,37.6783,6,14,3,3,0,2,2,11,1,1,0,1,2,5,35,19,120.7,127.1,104.4,106,30.6,2.8,0.265,111.32,85,0.21,4.22,2.85,2,3,5,115,2,0,82,5,6,8,17,3,4,7,9,14,0,17,0,5,14,1,5,3,3,32.35,0,1,2,2,2,9,11,4,1,0,4,0,3,4,16,19,128.5,131.4,101.9,105.8,22.2,5.6,0.183,106.27,70,0.158,4.34,2.5,9,14,22,69,0,0,55,5,...,0,0,0,0,0,0,0,0,1,150,100,100,100,0,0,0,107.06,3,0,3.74,0.06,0,0,0,2,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
21546,2018-19,ATL,ATL,DAL,0021800052,2018-10-24,ATL vs. DAL,1,1,34.23,5,9,3,4,0,4,4,3,2,0,0,0,6,3,24,5,100.8,100,92.5,92.4,13.6,9.1,0.22,109.57,78,0.129,4.33,2.61,0,5,5,50,0,0,30,0,2,8,13,2,2,3,6,6,4,15,12,0,1,2,0,5,0,3

In [46]:
test = wide_df.loc[wide_df['SEASON_YEAR'] == '2010-11']
test = test.loc[test['TEAM_ABBREVIATION'] == 'OKC'].sort_values('GAME_DATE')
test.iloc[:, 9:] = test.iloc[:, 9:].shift(1).rolling(2, min_periods=1).mean()
test['WL'] = test['WL'].shift(1).rolling(5, min_periods=1).mean()

# test.loc[test['GAME_DATE'] == '2010-12-25']

test.head(20)

,SEASON_YEAR,TEAM_ABBREVIATION,home_team_abbr,away_team_abbr,GAME_ID,GAME_DATE,MATCHUP,WL,HOME_GAME,P1_MIN,P1_FG3M,P1_FG3A,P1_FTM,P1_FTA,P1_OREB,P1_DREB,P1_REB,P1_AST,P1_TOV,P1_STL,P1_BLK,P1_BLKA,P1_PF,P1_PFD,P1_PTS,P1_PLUS_MINUS,P1_E_OFF_RATING,P1_OFF_RATING,P1_E_DEF_RATING,P1_DEF_RATING,P1_AST_RATIO,P1_TM_TOV_PCT,P1_E_USG_PCT,P1_E_PACE,P1_POSS,P1_PIE,P1_SPD,P1_DIST,P1_ORBC,P1_DRBC,P1_RBC,P1_TCHS,P1_SAST,P1_FTAST,P1_PASS,P1_CFGM,P1_CFGA,P1_UFGM,P1_UFGA,P1_DFGM,P1_DFGA,P1_FG2M,P1_FG2A,P1_PTS_2PT,P1_PTS_2PT_MR,P1_PTS_3PT,P1_PTS_FB,P1_PTS_OFF_TOV,P1_PTS_PAINT,P1_AST_2PM,P1_UAST_2PM,P1_AST_3PM,P1_UAST_3PM,P2_MIN,P2_FG3M,P2_FG3A,P2_FTM,P2_FTA,P2_OREB,P2_DREB,P2_REB,P2_AST,P2_TOV,P2_STL,P2_BLK,P2_BLKA,P2_PF,P2_PFD,P2_PTS,P2_PLUS_MINUS,P2_E_OFF_RATING,P2_OFF_RATING,P2_E_DEF_RATING,P2_DEF_RATING,P2_AST_RATIO,P2_TM_TOV_PCT,P2_E_USG_PCT,P2_E_PACE,P2_POSS,P2_PIE,P2_SPD,P2_DIST,P2_ORBC,P2_DRBC,P2_RBC,P2_TCHS,P2_SAST,P2_FTAST,P2_PASS,P2_CFGM,...,P11_AST,P11_TOV,P11_STL,P11_BLK,P11_BLKA,P11_PF,P11_PFD,P11_PTS,P11_PLUS_MINUS,P11_E_OFF_RATING,P11_OFF_RATING,P11_E_DEF_RATING,P11_DEF_RATING,P11_AST_RATIO,P11_TM_TOV_PCT,P11_E_USG_PCT,P11_E_PACE,P11_POSS,P11_PIE,P11_SPD,P11_DIST,P11_ORBC,P11_DRBC,P11_RBC,P11_TCHS,P11_SAST,P11_FTAST,P11_PASS,P11_CFGM,P11_CFGA,P11_UFGM,P11_UFGA,P11_DFGM,P11_DFGA,P11_FG2M,P11_FG2A,P11_PTS_2PT,P11_PTS_2PT_MR,P11_PTS_3PT,P11_PTS_FB,P11_PTS_OFF_TOV,P11_PTS_PAINT,P11_AST_2PM,P11_UAST_2PM,P11_AST_3PM,P11_UAST_3PM,P12_MIN,P12_FG3M,P12_FG3A,P12_FTM,P12_FTA,P12_OREB,P12_DREB,P12_REB,P12_AST,P12_TOV,P12_STL,P12_BLK,P12_BLKA,P12_PF,P12_PFD,P12_PTS,P12_PLUS_MINUS,P12_E_OFF_RATING,P12_OFF_RATING,P12_E_DEF_RATING,P12_DEF_RATING,P12_AST_RATIO,P12_TM_TOV_PCT,P12_E_USG_PCT,P12_E_PACE,P12_POSS,P12_PIE,P12_SPD,P12_DIST,P12_ORBC,P12_DRBC,P12_RBC,P12_TCHS,P12_SAST,P12_FTAST,P12_PASS,P12_CFGM,P12_CFGA,P12_UFGM,P12_UFGA,P12_DFGM,P12_DFGA,P12_FG2M,P12_FG2A,P12_PTS_2PT,P12_PTS_2PT_MR,P12_PTS_3PT,P12_PTS_FB,P12_PTS_OFF_TOV,P12_PTS_PAINT,P12_AST_2PM,P12_UAST_2PM,P12_AST_3PM,P12_UAST_3PM
2430,2010-11,OKC,OKC,CHI,0021000011,2010-10-27,OKC vs. CHI,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2401,2010-11,OKC,DET,OKC,0021000024,2010-10-29,OKC @ DET,1.000000,0,41.170000,1.0,5.0,11.0,13.0,1.0,6.0,7.0,3.0,6.0,3.0,0.0,2.0,3.0,8.0,30.0,15.0,109.00,111.60,91.20,93.10,7.70,15.40,0.3710,103.090,86.0,0.1170,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,19.0,15.0,3.0,3.0,6.0,2.0,12.0,4.0,4.0,1.0,0.0,39.883333,2.0,5.0,5.0,9.0,1.0,3.0,4.0,0.0,1.0,1.0,1.0,1.0,2.0,5.0,21.0,4.0,100.90,103.70,96.50,98.80,0.000000e+00,4.50,0.2360,101.220,82.0,0.0660,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0000,0.000,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2376,2010-11,OKC,OKC,UTA,0021000042,2010-10-31,OKC vs. UTA,1.000000,1,41.575833,1.0,6.5,8.5,10.5,0.5,5.5,6.0,1.5,4.5,3.0,0.5,1.0,2.5,6.5,25.5,11.5,109.95,113.55,99.20,100.15,3.85,13.70,0.3065,100.285,85.0,0.1010,0.0,0.0,0.0,0.0,0.0,0

In [4]:
def average_last_n(df = wide_df, n=10):
    averaged_data = []
    seasons = df['SEASON_YEAR'].unique()
    teams = df['TEAM_ABBREVIATION'].unique()
    for season in seasons:
        season_df = df.loc[df['SEASON_YEAR'] == season]
        for team in teams:
            team_df = season_df.loc[(season_df['TEAM_ABBREVIATION'] == team)].sort_values('GAME_DATE')
            team_df.iloc[:, 9:] = team_df.iloc[:, 9:].shift(1).rolling(n, min_periods=5).mean() 
            team_df['RECORD'] = team_df['WL'].shift(1).expanding().mean()
            averaged_data.append(team_df)
            
    averaged_df = pd.concat(averaged_data)
    return averaged_df


average_df = average_last_n(df=wide_df, n=10)

            

In [5]:
average_df

,SEASON_YEAR,TEAM_ABBREVIATION,home_team_abbr,away_team_abbr,GAME_ID,GAME_DATE,MATCHUP,WL,HOME_GAME,P1_MIN,P1_FG3M,P1_FG3A,P1_FTM,P1_FTA,P1_OREB,P1_DREB,P1_REB,P1_AST,P1_TOV,P1_STL,P1_BLK,P1_BLKA,P1_PF,P1_PFD,P1_PTS,P1_PLUS_MINUS,P1_E_OFF_RATING,P1_OFF_RATING,P1_E_DEF_RATING,P1_DEF_RATING,P1_AST_RATIO,P1_TM_TOV_PCT,P1_E_USG_PCT,P1_E_PACE,P1_POSS,P1_PIE,P1_SPD,P1_DIST,P1_ORBC,P1_DRBC,P1_RBC,P1_TCHS,P1_SAST,P1_FTAST,P1_PASS,P1_CFGM,P1_CFGA,P1_UFGM,P1_UFGA,P1_DFGM,P1_DFGA,P1_FG2M,P1_FG2A,P1_PTS_2PT,P1_PTS_2PT_MR,P1_PTS_3PT,P1_PTS_FB,P1_PTS_OFF_TOV,P1_PTS_PAINT,P1_AST_2PM,P1_UAST_2PM,P1_AST_3PM,P1_UAST_3PM,P2_MIN,P2_FG3M,P2_FG3A,P2_FTM,P2_FTA,P2_OREB,P2_DREB,P2_REB,P2_AST,P2_TOV,P2_STL,P2_BLK,P2_BLKA,P2_PF,P2_PFD,P2_PTS,P2_PLUS_MINUS,P2_E_OFF_RATING,P2_OFF_RATING,P2_E_DEF_RATING,P2_DEF_RATING,P2_AST_RATIO,P2_TM_TOV_PCT,P2_E_USG_PCT,P2_E_PACE,P2_POSS,P2_PIE,P2_SPD,P2_DIST,P2_ORBC,P2_DRBC,P2_RBC,P2_TCHS,P2_SAST,P2_FTAST,P2_PASS,P2_CFGM,...,P11_TOV,P11_STL,P11_BLK,P11_BLKA,P11_PF,P11_PFD,P11_PTS,P11_PLUS_MINUS,P11_E_OFF_RATING,P11_OFF_RATING,P11_E_DEF_RATING,P11_DEF_RATING,P11_AST_RATIO,P11_TM_TOV_PCT,P11_E_USG_PCT,P11_E_PACE,P11_POSS,P11_PIE,P11_SPD,P11_DIST,P11_ORBC,P11_DRBC,P11_RBC,P11_TCHS,P11_SAST,P11_FTAST,P11_PASS,P11_CFGM,P11_CFGA,P11_UFGM,P11_UFGA,P11_DFGM,P11_DFGA,P11_FG2M,P11_FG2A,P11_PTS_2PT,P11_PTS_2PT_MR,P11_PTS_3PT,P11_PTS_FB,P11_PTS_OFF_TOV,P11_PTS_PAINT,P11_AST_2PM,P11_UAST_2PM,P11_AST_3PM,P11_UAST_3PM,P12_MIN,P12_FG3M,P12_FG3A,P12_FTM,P12_FTA,P12_OREB,P12_DREB,P12_REB,P12_AST,P12_TOV,P12_STL,P12_BLK,P12_BLKA,P12_PF,P12_PFD,P12_PTS,P12_PLUS_MINUS,P12_E_OFF_RATING,P12_OFF_RATING,P12_E_DEF_RATING,P12_DEF_RATING,P12_AST_RATIO,P12_TM_TOV_PCT,P12_E_USG_PCT,P12_E_PACE,P12_POSS,P12_PIE,P12_SPD,P12_DIST,P12_ORBC,P12_DRBC,P12_RBC,P12_TCHS,P12_SAST,P12_FTAST,P12_PASS,P12_CFGM,P12_CFGA,P12_UFGM,P12_UFGA,P12_DFGM,P12_DFGA,P12_FG2M,P12_FG2A,P12_PTS_2PT,P12_PTS_2PT_MR,P12_PTS_3PT,P12_PTS_FB,P12_PTS_OFF_TOV,P12_PTS_PAINT,P12_AST_2PM,P12_UAST_2PM,P12_AST_3PM,P12_UAST_3PM,RECORD
2430,2010-11,OKC,OKC,CHI,21000011,2010-10-27,OKC vs. CHI,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2401,2010-11,OKC,DET,OKC,21000024,2010-10-29,OKC @ DET,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
2376,2010-11,OKC,OKC,UTA,21000042,2010-10-31,OKC vs. UTA,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [75]:
test = average_df.loc[average_df['SEASON_YEAR'] == '2010-11']

test.loc[test['TEAM_ABBREVIATION'] == 'OKC'].head(10)

,SEASON_YEAR,TEAM_ABBREVIATION,home_team_abbr,away_team_abbr,GAME_ID,GAME_DATE,MATCHUP,WL,HOME_GAME,P1_MIN,P1_FG3M,P1_FG3A,P1_FTM,P1_FTA,P1_OREB,P1_DREB,P1_REB,P1_AST,P1_TOV,P1_STL,P1_BLK,P1_BLKA,P1_PF,P1_PFD,P1_PTS,P1_PLUS_MINUS,P1_E_OFF_RATING,P1_OFF_RATING,P1_E_DEF_RATING,P1_DEF_RATING,P1_AST_RATIO,P1_TM_TOV_PCT,P1_E_USG_PCT,P1_E_PACE,P1_POSS,P1_PIE,P1_SPD,P1_DIST,P1_ORBC,P1_DRBC,P1_RBC,P1_TCHS,P1_SAST,P1_FTAST,P1_PASS,P1_CFGM,P1_CFGA,P1_UFGM,P1_UFGA,P1_DFGM,P1_DFGA,P1_FG2M,P1_FG2A,P1_PTS_2PT,P1_PTS_2PT_MR,P1_PTS_3PT,P1_PTS_FB,P1_PTS_OFF_TOV,P1_PTS_PAINT,P1_AST_2PM,P1_UAST_2PM,P1_AST_3PM,P1_UAST_3PM,P2_MIN,P2_FG3M,P2_FG3A,P2_FTM,P2_FTA,P2_OREB,P2_DREB,P2_REB,P2_AST,P2_TOV,P2_STL,P2_BLK,P2_BLKA,P2_PF,P2_PFD,P2_PTS,P2_PLUS_MINUS,P2_E_OFF_RATING,P2_OFF_RATING,P2_E_DEF_RATING,P2_DEF_RATING,P2_AST_RATIO,P2_TM_TOV_PCT,P2_E_USG_PCT,P2_E_PACE,P2_POSS,P2_PIE,P2_SPD,P2_DIST,P2_ORBC,P2_DRBC,P2_RBC,P2_TCHS,P2_SAST,P2_FTAST,P2_PASS,P2_CFGM,...,P11_TOV,P11_STL,P11_BLK,P11_BLKA,P11_PF,P11_PFD,P11_PTS,P11_PLUS_MINUS,P11_E_OFF_RATING,P11_OFF_RATING,P11_E_DEF_RATING,P11_DEF_RATING,P11_AST_RATIO,P11_TM_TOV_PCT,P11_E_USG_PCT,P11_E_PACE,P11_POSS,P11_PIE,P11_SPD,P11_DIST,P11_ORBC,P11_DRBC,P11_RBC,P11_TCHS,P11_SAST,P11_FTAST,P11_PASS,P11_CFGM,P11_CFGA,P11_UFGM,P11_UFGA,P11_DFGM,P11_DFGA,P11_FG2M,P11_FG2A,P11_PTS_2PT,P11_PTS_2PT_MR,P11_PTS_3PT,P11_PTS_FB,P11_PTS_OFF_TOV,P11_PTS_PAINT,P11_AST_2PM,P11_UAST_2PM,P11_AST_3PM,P11_UAST_3PM,P12_MIN,P12_FG3M,P12_FG3A,P12_FTM,P12_FTA,P12_OREB,P12_DREB,P12_REB,P12_AST,P12_TOV,P12_STL,P12_BLK,P12_BLKA,P12_PF,P12_PFD,P12_PTS,P12_PLUS_MINUS,P12_E_OFF_RATING,P12_OFF_RATING,P12_E_DEF_RATING,P12_DEF_RATING,P12_AST_RATIO,P12_TM_TOV_PCT,P12_E_USG_PCT,P12_E_PACE,P12_POSS,P12_PIE,P12_SPD,P12_DIST,P12_ORBC,P12_DRBC,P12_RBC,P12_TCHS,P12_SAST,P12_FTAST,P12_PASS,P12_CFGM,P12_CFGA,P12_UFGM,P12_UFGA,P12_DFGM,P12_DFGA,P12_FG2M,P12_FG2A,P12_PTS_2PT,P12_PTS_2PT_MR,P12_PTS_3PT,P12_PTS_FB,P12_PTS_OFF_TOV,P12_PTS_PAINT,P12_AST_2PM,P12_UAST_2PM,P12_AST_3PM,P12_UAST_3PM,RECORD
2430,2010-11,OKC,OKC,CHI,0021000011,2010-10-27,OKC vs. CHI,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2401,2010-11,OKC,DET,OKC,0021000024,2010-10-29,OKC @ DET,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
2376,2010-11,OKC,OKC,UTA,0021000042,2010-10-31,OKC vs. UTA,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [18]:


home_games = average_df.loc[average_df['HOME_GAME'] == 1]

cols = home_games.columns[:7].tolist()

home_cols = ['home_'+x for x in home_games.columns[7:]]
cols.extend(home_cols)
cols
home_games.columns = cols

away_games = average_df.loc[average_df['HOME_GAME'] == 0]

away_games = away_games.drop(columns = ['SEASON_YEAR', 'TEAM_ABBREVIATION', 
                            'home_team_abbr', 'away_team_abbr',
                            'GAME_DATE', 'MATCHUP'])

away_games
cols = ['GAME_ID']
away_cols = ['away_'+x for x in away_games.columns[1:]]
cols.extend(away_cols)



away_games.columns = cols

matchup_average = pd.merge(home_games, away_games, on='GAME_ID', suffixes=['', ''])

matchup_average['GAME_DATE'] = pd.to_datetime(matchup_average['GAME_DATE'])

matchup_average

,SEASON_YEAR,TEAM_ABBREVIATION,home_team_abbr,away_team_abbr,GAME_ID,GAME_DATE,MATCHUP,home_WL,home_HOME_GAME,home_P1_MIN,home_P1_FG3M,home_P1_FG3A,home_P1_FTM,home_P1_FTA,home_P1_OREB,home_P1_DREB,home_P1_REB,home_P1_AST,home_P1_TOV,home_P1_STL,home_P1_BLK,home_P1_BLKA,home_P1_PF,home_P1_PFD,home_P1_PTS,home_P1_PLUS_MINUS,home_P1_E_OFF_RATING,home_P1_OFF_RATING,home_P1_E_DEF_RATING,home_P1_DEF_RATING,home_P1_AST_RATIO,home_P1_TM_TOV_PCT,home_P1_E_USG_PCT,home_P1_E_PACE,home_P1_POSS,home_P1_PIE,home_P1_SPD,home_P1_DIST,home_P1_ORBC,home_P1_DRBC,home_P1_RBC,home_P1_TCHS,home_P1_SAST,home_P1_FTAST,home_P1_PASS,home_P1_CFGM,home_P1_CFGA,home_P1_UFGM,home_P1_UFGA,home_P1_DFGM,home_P1_DFGA,home_P1_FG2M,home_P1_FG2A,home_P1_PTS_2PT,home_P1_PTS_2PT_MR,home_P1_PTS_3PT,home_P1_PTS_FB,home_P1_PTS_OFF_TOV,home_P1_PTS_PAINT,home_P1_AST_2PM,home_P1_UAST_2PM,home_P1_AST_3PM,home_P1_UAST_3PM,home_P2_MIN,home_P2_FG3M,home_P2_FG3A,home_P2_FTM,home_P2_FTA,home_P2_OREB,home_P2_DREB,home_P2_REB,home_P2_AST,home_P2_TOV,home_P2_STL,home_P2_BLK,home_P2_BLKA,home_P2_PF,home_P2_PFD,home_P2_PTS,home_P2_PLUS_MINUS,home_P2_E_OFF_RATING,home_P2_OFF_RATING,home_P2_E_DEF_RATING,home_P2_DEF_RATING,home_P2_AST_RATIO,home_P2_TM_TOV_PCT,home_P2_E_USG_PCT,home_P2_E_PACE,home_P2_POSS,home_P2_PIE,home_P2_SPD,home_P2_DIST,home_P2_ORBC,home_P2_DRBC,home_P2_RBC,home_P2_TCHS,home_P2_SAST,home_P2_FTAST,home_P2_PASS,home_P2_CFGM,...,away_P11_TOV,away_P11_STL,away_P11_BLK,away_P11_BLKA,away_P11_PF,away_P11_PFD,away_P11_PTS,away_P11_PLUS_MINUS,away_P11_E_OFF_RATING,away_P11_OFF_RATING,away_P11_E_DEF_RATING,away_P11_DEF_RATING,away_P11_AST_RATIO,away_P11_TM_TOV_PCT,away_P11_E_USG_PCT,away_P11_E_PACE,away_P11_POSS,away_P11_PIE,away_P11_SPD,away_P11_DIST,away_P11_ORBC,away_P11_DRBC,away_P11_RBC,away_P11_TCHS,away_P11_SAST,away_P11_FTAST,away_P11_PASS,away_P11_CFGM,away_P11_CFGA,away_P11_UFGM,away_P11_UFGA,away_P11_DFGM,away_P11_DFGA,away_P11_FG2M,away_P11_FG2A,away_P11_PTS_2PT,away_P11_PTS_2PT_MR,away_P11_PTS_3PT,away_P11_PTS_FB,away_P11_PTS_OFF_TOV,away_P11_PTS_PAINT,away_P11_AST_2PM,away_P11_UAST_2PM,away_P11_AST_3PM,away_P11_UAST_3PM,away_P12_MIN,away_P12_FG3M,away_P12_FG3A,away_P12_FTM,away_P12_FTA,away_P12_OREB,away_P12_DREB,away_P12_REB,away_P12_AST,away_P12_TOV,away_P12_STL,away_P12_BLK,away_P12_BLKA,away_P12_PF,away_P12_PFD,away_P12_PTS,away_P12_PLUS_MINUS,away_P12_E_OFF_RATING,away_P12_OFF_RATING,away_P12_E_DEF_RATING,away_P12_DEF_RATING,away_P12_AST_RATIO,away_P12_TM_TOV_PCT,away_P12_E_USG_PCT,away_P12_E_PACE,away_P12_POSS,away_P12_PIE,away_P12_SPD,away_P12_DIST,away_P12_ORBC,away_P12_DRBC,away_P12_RBC,away_P12_TCHS,away_P12_SAST,away_P12_FTAST,away_P12_PASS,away_P12_CFGM,away_P12_CFGA,away_P12_UFGM,away_P12_UFGA,away_P12_DFGM,away_P12_DFGA,away_P12_FG2M,away_P12_FG2A,away_P12_PTS_2PT,away_P12_PTS_2PT_MR,away_P12_PTS_3PT,away_P12_PTS_FB,away_P12_PTS_OFF_TOV,away_P12_PTS_PAINT,away_P12_AST_2PM,away_P12_UAST_2PM,away_P12_AST_3PM,away_P12_UAST_3PM,away_RECORD
0,2010-11,OKC,OKC,CHI,21000011,2010-10-27,OKC vs. CHI,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010-11,OKC,OKC,UTA,21000042,2010-10-31,OKC vs. UTA,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [7]:
test = matchup_average.loc[matchup_average['SEASON_YEAR'] == '2010-11'].sort_values('GAME_DATE')
test.loc[(test['home_team_abbr'] == 'OKC')| (test['away_team_abbr'] == 'OKC')]

,SEASON_YEAR,TEAM_ABBREVIATION,home_team_abbr,away_team_abbr,GAME_ID,GAME_DATE,MATCHUP,home_WL,home_HOME_GAME,home_P1_MIN,home_P1_FG3M,home_P1_FG3A,home_P1_FTM,home_P1_FTA,home_P1_OREB,home_P1_DREB,home_P1_REB,home_P1_AST,home_P1_TOV,home_P1_STL,home_P1_BLK,home_P1_BLKA,home_P1_PF,home_P1_PFD,home_P1_PTS,home_P1_PLUS_MINUS,home_P1_E_OFF_RATING,home_P1_OFF_RATING,home_P1_E_DEF_RATING,home_P1_DEF_RATING,home_P1_AST_RATIO,home_P1_TM_TOV_PCT,home_P1_E_USG_PCT,home_P1_E_PACE,home_P1_POSS,home_P1_PIE,home_P1_SPD,home_P1_DIST,home_P1_ORBC,home_P1_DRBC,home_P1_RBC,home_P1_TCHS,home_P1_SAST,home_P1_FTAST,home_P1_PASS,home_P1_CFGM,home_P1_CFGA,home_P1_UFGM,home_P1_UFGA,home_P1_DFGM,home_P1_DFGA,home_P1_FG2M,home_P1_FG2A,home_P1_PTS_2PT,home_P1_PTS_2PT_MR,home_P1_PTS_3PT,home_P1_PTS_FB,home_P1_PTS_OFF_TOV,home_P1_PTS_PAINT,home_P1_AST_2PM,home_P1_UAST_2PM,home_P1_AST_3PM,home_P1_UAST_3PM,home_P2_MIN,home_P2_FG3M,home_P2_FG3A,home_P2_FTM,home_P2_FTA,home_P2_OREB,home_P2_DREB,home_P2_REB,home_P2_AST,home_P2_TOV,home_P2_STL,home_P2_BLK,home_P2_BLKA,home_P2_PF,home_P2_PFD,home_P2_PTS,home_P2_PLUS_MINUS,home_P2_E_OFF_RATING,home_P2_OFF_RATING,home_P2_E_DEF_RATING,home_P2_DEF_RATING,home_P2_AST_RATIO,home_P2_TM_TOV_PCT,home_P2_E_USG_PCT,home_P2_E_PACE,home_P2_POSS,home_P2_PIE,home_P2_SPD,home_P2_DIST,home_P2_ORBC,home_P2_DRBC,home_P2_RBC,home_P2_TCHS,home_P2_SAST,home_P2_FTAST,home_P2_PASS,home_P2_CFGM,...,away_P11_TOV,away_P11_STL,away_P11_BLK,away_P11_BLKA,away_P11_PF,away_P11_PFD,away_P11_PTS,away_P11_PLUS_MINUS,away_P11_E_OFF_RATING,away_P11_OFF_RATING,away_P11_E_DEF_RATING,away_P11_DEF_RATING,away_P11_AST_RATIO,away_P11_TM_TOV_PCT,away_P11_E_USG_PCT,away_P11_E_PACE,away_P11_POSS,away_P11_PIE,away_P11_SPD,away_P11_DIST,away_P11_ORBC,away_P11_DRBC,away_P11_RBC,away_P11_TCHS,away_P11_SAST,away_P11_FTAST,away_P11_PASS,away_P11_CFGM,away_P11_CFGA,away_P11_UFGM,away_P11_UFGA,away_P11_DFGM,away_P11_DFGA,away_P11_FG2M,away_P11_FG2A,away_P11_PTS_2PT,away_P11_PTS_2PT_MR,away_P11_PTS_3PT,away_P11_PTS_FB,away_P11_PTS_OFF_TOV,away_P11_PTS_PAINT,away_P11_AST_2PM,away_P11_UAST_2PM,away_P11_AST_3PM,away_P11_UAST_3PM,away_P12_MIN,away_P12_FG3M,away_P12_FG3A,away_P12_FTM,away_P12_FTA,away_P12_OREB,away_P12_DREB,away_P12_REB,away_P12_AST,away_P12_TOV,away_P12_STL,away_P12_BLK,away_P12_BLKA,away_P12_PF,away_P12_PFD,away_P12_PTS,away_P12_PLUS_MINUS,away_P12_E_OFF_RATING,away_P12_OFF_RATING,away_P12_E_DEF_RATING,away_P12_DEF_RATING,away_P12_AST_RATIO,away_P12_TM_TOV_PCT,away_P12_E_USG_PCT,away_P12_E_PACE,away_P12_POSS,away_P12_PIE,away_P12_SPD,away_P12_DIST,away_P12_ORBC,away_P12_DRBC,away_P12_RBC,away_P12_TCHS,away_P12_SAST,away_P12_FTAST,away_P12_PASS,away_P12_CFGM,away_P12_CFGA,away_P12_UFGM,away_P12_UFGA,away_P12_DFGM,away_P12_DFGA,away_P12_FG2M,away_P12_FG2A,away_P12_PTS_2PT,away_P12_PTS_2PT_MR,away_P12_PTS_3PT,away_P12_PTS_FB,away_P12_PTS_OFF_TOV,away_P12_PTS_PAINT,away_P12_AST_2PM,away_P12_UAST_2PM,away_P12_AST_3PM,away_P12_UAST_3PM,away_RECORD
0,2010-11,OKC,OKC,CHI,21000011,2010-10-27,OKC vs. CHI,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
451,2010-11,DET,DET,OKC,21000024,2010-10-29,DET vs. OKC,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [8]:
matchup_average

,SEASON_YEAR,TEAM_ABBREVIATION,home_team_abbr,away_team_abbr,GAME_ID,GAME_DATE,MATCHUP,home_WL,home_HOME_GAME,home_P1_MIN,home_P1_FG3M,home_P1_FG3A,home_P1_FTM,home_P1_FTA,home_P1_OREB,home_P1_DREB,home_P1_REB,home_P1_AST,home_P1_TOV,home_P1_STL,home_P1_BLK,home_P1_BLKA,home_P1_PF,home_P1_PFD,home_P1_PTS,home_P1_PLUS_MINUS,home_P1_E_OFF_RATING,home_P1_OFF_RATING,home_P1_E_DEF_RATING,home_P1_DEF_RATING,home_P1_AST_RATIO,home_P1_TM_TOV_PCT,home_P1_E_USG_PCT,home_P1_E_PACE,home_P1_POSS,home_P1_PIE,home_P1_SPD,home_P1_DIST,home_P1_ORBC,home_P1_DRBC,home_P1_RBC,home_P1_TCHS,home_P1_SAST,home_P1_FTAST,home_P1_PASS,home_P1_CFGM,home_P1_CFGA,home_P1_UFGM,home_P1_UFGA,home_P1_DFGM,home_P1_DFGA,home_P1_FG2M,home_P1_FG2A,home_P1_PTS_2PT,home_P1_PTS_2PT_MR,home_P1_PTS_3PT,home_P1_PTS_FB,home_P1_PTS_OFF_TOV,home_P1_PTS_PAINT,home_P1_AST_2PM,home_P1_UAST_2PM,home_P1_AST_3PM,home_P1_UAST_3PM,home_P2_MIN,home_P2_FG3M,home_P2_FG3A,home_P2_FTM,home_P2_FTA,home_P2_OREB,home_P2_DREB,home_P2_REB,home_P2_AST,home_P2_TOV,home_P2_STL,home_P2_BLK,home_P2_BLKA,home_P2_PF,home_P2_PFD,home_P2_PTS,home_P2_PLUS_MINUS,home_P2_E_OFF_RATING,home_P2_OFF_RATING,home_P2_E_DEF_RATING,home_P2_DEF_RATING,home_P2_AST_RATIO,home_P2_TM_TOV_PCT,home_P2_E_USG_PCT,home_P2_E_PACE,home_P2_POSS,home_P2_PIE,home_P2_SPD,home_P2_DIST,home_P2_ORBC,home_P2_DRBC,home_P2_RBC,home_P2_TCHS,home_P2_SAST,home_P2_FTAST,home_P2_PASS,home_P2_CFGM,...,away_P11_TOV,away_P11_STL,away_P11_BLK,away_P11_BLKA,away_P11_PF,away_P11_PFD,away_P11_PTS,away_P11_PLUS_MINUS,away_P11_E_OFF_RATING,away_P11_OFF_RATING,away_P11_E_DEF_RATING,away_P11_DEF_RATING,away_P11_AST_RATIO,away_P11_TM_TOV_PCT,away_P11_E_USG_PCT,away_P11_E_PACE,away_P11_POSS,away_P11_PIE,away_P11_SPD,away_P11_DIST,away_P11_ORBC,away_P11_DRBC,away_P11_RBC,away_P11_TCHS,away_P11_SAST,away_P11_FTAST,away_P11_PASS,away_P11_CFGM,away_P11_CFGA,away_P11_UFGM,away_P11_UFGA,away_P11_DFGM,away_P11_DFGA,away_P11_FG2M,away_P11_FG2A,away_P11_PTS_2PT,away_P11_PTS_2PT_MR,away_P11_PTS_3PT,away_P11_PTS_FB,away_P11_PTS_OFF_TOV,away_P11_PTS_PAINT,away_P11_AST_2PM,away_P11_UAST_2PM,away_P11_AST_3PM,away_P11_UAST_3PM,away_P12_MIN,away_P12_FG3M,away_P12_FG3A,away_P12_FTM,away_P12_FTA,away_P12_OREB,away_P12_DREB,away_P12_REB,away_P12_AST,away_P12_TOV,away_P12_STL,away_P12_BLK,away_P12_BLKA,away_P12_PF,away_P12_PFD,away_P12_PTS,away_P12_PLUS_MINUS,away_P12_E_OFF_RATING,away_P12_OFF_RATING,away_P12_E_DEF_RATING,away_P12_DEF_RATING,away_P12_AST_RATIO,away_P12_TM_TOV_PCT,away_P12_E_USG_PCT,away_P12_E_PACE,away_P12_POSS,away_P12_PIE,away_P12_SPD,away_P12_DIST,away_P12_ORBC,away_P12_DRBC,away_P12_RBC,away_P12_TCHS,away_P12_SAST,away_P12_FTAST,away_P12_PASS,away_P12_CFGM,away_P12_CFGA,away_P12_UFGM,away_P12_UFGA,away_P12_DFGM,away_P12_DFGA,away_P12_FG2M,away_P12_FG2A,away_P12_PTS_2PT,away_P12_PTS_2PT_MR,away_P12_PTS_3PT,away_P12_PTS_FB,away_P12_PTS_OFF_TOV,away_P12_PTS_PAINT,away_P12_AST_2PM,away_P12_UAST_2PM,away_P12_AST_3PM,away_P12_UAST_3PM,away_RECORD
0,2010-11,OKC,OKC,CHI,21000011,2010-10-27,OKC vs. CHI,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010-11,OKC,OKC,UTA,21000042,2010-10-31,OKC vs. UTA,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

## Clean and Process Betting Data

In [9]:
# read betting data
betting_data = pd.read_csv('./data/nba_betting_data_2010_present.csv')

betting_data = betting_data.dropna()

In [19]:
def clean_bet_data(df = betting_data):
    df['date'] = pd.to_datetime(df['date'])
    df['home_team_abbr'] = df['home_team_abbr'].replace({'NY':'NYK',
                                                                            'GS':'GSW',
                                                                            'SA':'SAS',
                                                                            'BK':'BKN',
                                                                            'NO':'NOP',
                                                                            'PHO':'PHX'
                                                                                        }
                                                                             )
    df['away_team_abbr'] = df['away_team_abbr'].replace({'NY':'NYK',
                                                                            'GS':'GSW',
                                                                            'SA':'SAS',
                                                                            'BK':'BKN',
                                                                            'NO':'NOP',
                                                                            'PHO':'PHX'}
                                                                           )

    teams = df['home_team_abbr'].unique()

    df = df.sort_values(['date'])
    df['rest'] = np.nan
    for team in teams:
        team_data = df.loc[(df['home_team_abbr'] == team) | (df['away_team_abbr'] == team)]
        team_data['rest'] = (team_data['date'] - team_data['date'].shift(1)) / np.timedelta64(1, 'D')
        df.loc[(df['home_team_abbr'] == team) | (df['away_team_abbr'] == team), 'rest'] = team_data['rest']


    df['point_diff'] = df['home_score'] - df['away_score']
    df['point_total'] = df['home_score'] + df['away_score']
    df['covered_spread'] = (df['home_score'] + df['spread'] > df['away_score']).astype(int)
    df['over'] = (df['point_total'] > df['total']).astype(int)
    
    relevant_betting = df[['date', 'home_team_abbr',  'away_team_abbr',
                           'home_score', 'away_score', 'point_diff', 
                           'rest', 'point_total', 'spread', 'total',
                          'covered_spread', 'over', ]]

    
    return relevant_betting

In [20]:
betting_data = clean_bet_data(betting_data)

C:\Users\Jordan Nishimura\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [21]:
betting_data

,date,home_team_abbr,away_team_abbr,home_score,away_score,point_diff,rest,point_total,spread,total,covered_spread,over
1012,2011-12-25,NYK,BOS,106,104,2,NaN,210,-5.0,190.5,0,1
1013,2011-12-25,DAL,MIA,94,105,-11,NaN,199,4.5,188.5,0,1
1014,2011-12-25,LAL,CHI,87,88,-1,NaN,175,4.5,183.5,1,0
1015,2011-12-25,OKC,ORL,97,89,8,NaN,186,-7.5,194.5,1,0
1016,2011-12-25,GSW,LAC,86,105,-19,NaN,191,6.0,207.5,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
11933,2021-02-04,PHI,POR,105,121,-16,1.0,226,-10.0,223.0,0,1
11932,2021-02-04,DAL,GSW,116,147,-31,2.0,263,-4.0,229.5,0,1
11935,2021-02-04,LAL,DEN,114,93,21,4.0,207,-5.0,217.0,1,0
11934,2021-02-04,MEM,HOU,103,115,-12,1.0,218,-3.5,226.0,0,0


In [22]:
full_df = pd.merge(betting_data, matchup_average, left_on=['date', 'home_team_abbr', 'away_team_abbr'], right_on=['GAME_DATE', 'home_team_abbr', 'away_team_abbr'])


In [23]:
full_df

,date,home_team_abbr,away_team_abbr,home_score,away_score,point_diff,rest,point_total,spread,total,covered_spread,over,SEASON_YEAR,TEAM_ABBREVIATION,GAME_ID,GAME_DATE,MATCHUP,home_WL,home_HOME_GAME,home_P1_MIN,home_P1_FG3M,home_P1_FG3A,home_P1_FTM,home_P1_FTA,home_P1_OREB,home_P1_DREB,home_P1_REB,home_P1_AST,home_P1_TOV,home_P1_STL,home_P1_BLK,home_P1_BLKA,home_P1_PF,home_P1_PFD,home_P1_PTS,home_P1_PLUS_MINUS,home_P1_E_OFF_RATING,home_P1_OFF_RATING,home_P1_E_DEF_RATING,home_P1_DEF_RATING,home_P1_AST_RATIO,home_P1_TM_TOV_PCT,home_P1_E_USG_PCT,home_P1_E_PACE,home_P1_POSS,home_P1_PIE,home_P1_SPD,home_P1_DIST,home_P1_ORBC,home_P1_DRBC,home_P1_RBC,home_P1_TCHS,home_P1_SAST,home_P1_FTAST,home_P1_PASS,home_P1_CFGM,home_P1_CFGA,home_P1_UFGM,home_P1_UFGA,home_P1_DFGM,home_P1_DFGA,home_P1_FG2M,home_P1_FG2A,home_P1_PTS_2PT,home_P1_PTS_2PT_MR,home_P1_PTS_3PT,home_P1_PTS_FB,home_P1_PTS_OFF_TOV,home_P1_PTS_PAINT,home_P1_AST_2PM,home_P1_UAST_2PM,home_P1_AST_3PM,home_P1_UAST_3PM,home_P2_MIN,home_P2_FG3M,home_P2_FG3A,home_P2_FTM,home_P2_FTA,home_P2_OREB,home_P2_DREB,home_P2_REB,home_P2_AST,home_P2_TOV,home_P2_STL,home_P2_BLK,home_P2_BLKA,home_P2_PF,home_P2_PFD,home_P2_PTS,home_P2_PLUS_MINUS,home_P2_E_OFF_RATING,home_P2_OFF_RATING,home_P2_E_DEF_RATING,home_P2_DEF_RATING,home_P2_AST_RATIO,home_P2_TM_TOV_PCT,home_P2_E_USG_PCT,home_P2_E_PACE,home_P2_POSS,home_P2_PIE,...,away_P11_TOV,away_P11_STL,away_P11_BLK,away_P11_BLKA,away_P11_PF,away_P11_PFD,away_P11_PTS,away_P11_PLUS_MINUS,away_P11_E_OFF_RATING,away_P11_OFF_RATING,away_P11_E_DEF_RATING,away_P11_DEF_RATING,away_P11_AST_RATIO,away_P11_TM_TOV_PCT,away_P11_E_USG_PCT,away_P11_E_PACE,away_P11_POSS,away_P11_PIE,away_P11_SPD,away_P11_DIST,away_P11_ORBC,away_P11_DRBC,away_P11_RBC,away_P11_TCHS,away_P11_SAST,away_P11_FTAST,away_P11_PASS,away_P11_CFGM,away_P11_CFGA,away_P11_UFGM,away_P11_UFGA,away_P11_DFGM,away_P11_DFGA,away_P11_FG2M,away_P11_FG2A,away_P11_PTS_2PT,away_P11_PTS_2PT_MR,away_P11_PTS_3PT,away_P11_PTS_FB,away_P11_PTS_OFF_TOV,away_P11_PTS_PAINT,away_P11_AST_2PM,away_P11_UAST_2PM,away_P11_AST_3PM,away_P11_UAST_3PM,away_P12_MIN,away_P12_FG3M,away_P12_FG3A,away_P12_FTM,away_P12_FTA,away_P12_OREB,away_P12_DREB,away_P12_REB,away_P12_AST,away_P12_TOV,away_P12_STL,away_P12_BLK,away_P12_BLKA,away_P12_PF,away_P12_PFD,away_P12_PTS,away_P12_PLUS_MINUS,away_P12_E_OFF_RATING,away_P12_OFF_RATING,away_P12_E_DEF_RATING,away_P12_DEF_RATING,away_P12_AST_RATIO,away_P12_TM_TOV_PCT,away_P12_E_USG_PCT,away_P12_E_PACE,away_P12_POSS,away_P12_PIE,away_P12_SPD,away_P12_DIST,away_P12_ORBC,away_P12_DRBC,away_P12_RBC,away_P12_TCHS,away_P12_SAST,away_P12_FTAST,away_P12_PASS,away_P12_CFGM,away_P12_CFGA,away_P12_UFGM,away_P12_UFGA,away_P12_DFGM,away_P12_DFGA,away_P12_FG2M,away_P12_FG2A,away_P12_PTS_2PT,away_P12_PTS_2PT_MR,away_P12_PTS_3PT,away_P12_PTS_FB,away_P12_PTS_OFF_TOV,away_P12_PTS_PAINT,away_P12_AST_2PM,away_P12_UAST_2PM,away_P12_AST_3PM,away_P12_UAST_3PM,away_RECORD
0,2011-12-25,NYK,BOS,106,104,2,NaN,210,-5.0,190.5,0,1,2011-12,NYK,21100001,2011-12-25,NYK vs. BOS,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2011-12-25,DAL,MIA,94,105,-11,NaN,199,4.5,188.5,0,1,2011-12,DAL,21100002,2011-12-25,DAL vs. MIA,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [96]:
full_df['prev_cover'] = full_df['covered_spread'].shift(2)
full_df['prev2_cover'] = full_df['covered_spread'].shift(2)

In [103]:
print("original shape", full_df.shape)
full_df = full_df.dropna()
print("dropped_na", full_df.shape)
features = full_df.drop(columns=['date', 'home_team_abbr', 'away_team_abbr', 
                                 'home_score', 'away_score', 'point_diff', 
                                 'rest', 'point_total', 'spread', 'total',
                                 'covered_spread', 'over', 'SEASON_YEAR', 
                                 'TEAM_ABBREVIATION', 'GAME_ID', 'GAME_DATE',
                                 'MATCHUP', 'home_WL', 'home_HOME_GAME'])

features.shape

original shape (10951, 1321)
dropped_na (10161, 1321)


(10161, 1302)

In [119]:
y = full_df['point_diff']

In [116]:
features[['away_HOME_GAME', 'away_WL']]

,away_HOME_GAME,away_WL
65,0,0
76,0,0
81,0,0
82,0,1
84,0,0
...,...,...
10946,0,0
10947,0,1
10948,0,1
10949,0,1


## Removing Correlated Features

In [118]:
def remove_correlated_features(final_merged_df, threshold=1):
    """Removes correlated features from dataset
    INPUTS:
    final_merged_df
    threshold: float between 0 and 1 indicating how highly correlated features
    must be to be removed
    OUTPUT: final_merged_df with the correlated feature columns removed
    """
    corr = final_merged_df.select_dtypes('number').corr()

    correlated_features = set()
    for i in range(len(corr.columns)):
        for j in range(i):
            if abs(corr.iloc[i, j]) >= threshold:
                colname = corr.columns[i]
                correlated_features.add(colname)
    print("Correlated Features:", list(correlated_features))
    print("Num Features Removed ", len(correlated_features))
    final_merged_df.drop(columns=list(correlated_features), inplace=True)
    return final_merged_df

features = remove_correlated_features(features, threshold=1)

Correlated Features: ['prev2_cover']
Num Features Removed  1


In [117]:
from sklearn.linear_model import ElasticNet

fea

In [31]:
def clean_team_gamelog(team_gamelog = team_gamelog):
    df = team_gamelog.copy()
    df = df[['SEASON_YEAR', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID',
       'GAME_DATE', 'MATCHUP', 'WL', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M',
       'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST',
       'TOV', 'STL', 'BLK', 'BLKA', 'PF', 'PFD', 'PTS', 'PLUS_MINUS']]
    
    # Convert GAME_DATE to datetime so it is sortable
    df['GAME_DATE'] = pd.to_datetime(df['GAME_DATE'])
    
    # Sort by team and date
    df.sort_values(['TEAM_ABBREVIATION', 'GAME_DATE'], inplace=True)
    
    # Combine rows so that one instance shows a game with team stats and opponent stats
    full_gamelogs = pd.merge(df, df, how='inner', on='GAME_ID', suffixes=['', '_opp'])

    # Remove rows that were created where the team is playing itself
    full_gamelogs = full_gamelogs.loc[full_gamelogs['TEAM_ABBREVIATION'] != full_gamelogs['TEAM_ABBREVIATION_opp']]

    # Add column for HOME_GAME
    full_gamelogs['HOME_GAME'] = (full_gamelogs['MATCHUP'].str[:6] == (full_gamelogs['TEAM_ABBREVIATION'].str[:] + ' vs')).astype(int)
    
    full_gamelogs['WIN'] = (full_gamelogs['WL'] == 'W').astype(int)
    full_gamelogs.drop(columns='WL', inplace=True)
    
    full_gamelogs['point_diff'] = full_gamelogs['PLUS_MINUS']
    
    full_gamelogs.sort_values(['TEAM_ABBREVIATION', 'GAME_DATE'], inplace=True)
    
    full_gamelogs['prev_WIN'] = full_gamelogs['WIN'].shift(1)
    full_gamelogs['prev2_WIN'] = full_gamelogs['WIN'].shift(2)
    
    # Linearly dependent variables
    full_gamelogs = full_gamelogs.drop(columns=['FG_PCT', 'FG3_PCT', 'FT_PCT', 'REB',
                                               'FG_PCT_opp', 'FG3_PCT_opp', 'FT_PCT_opp', 'REB_opp',
                                               'BLK_opp', 'BLKA_opp', 'PF_opp', 'PFD_opp', 'PLUS_MINUS_opp',
                                               'MIN_opp'])
    
    return full_gamelogs
    
clean_gamelogs_2018 = clean_team_gamelog(team_gamelog)
clean_gamelogs_2018

,SEASON_YEAR,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,MIN,FGM,FGA,FG3M,FG3A,FTM,FTA,OREB,DREB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,SEASON_YEAR_opp,TEAM_ID_opp,TEAM_ABBREVIATION_opp,TEAM_NAME_opp,GAME_DATE_opp,MATCHUP_opp,WL_opp,FGM_opp,FGA_opp,FG3M_opp,FG3A_opp,FTM_opp,FTA_opp,OREB_opp,DREB_opp,AST_opp,TOV_opp,STL_opp,PTS_opp,HOME_GAME,WIN,point_diff,prev_WIN,prev2_WIN
1,2018-19,1610612737,ATL,Atlanta Hawks,21800007,2018-10-17,ATL @ NYK,48.0,41,90,10,36,15,18,7,38,19,24.0,7,3,6,30,23,107,-19.0,2018-19,1610612752,NYK,New York Knicks,2018-10-17,NYK vs. ATL,W,45,99,12,33,24,31,10,36,21,16.0,12,126,0,0,-19.0,NaN,NaN
5,2018-19,1610612737,ATL,Atlanta Hawks,21800020,2018-10-19,ATL @ MEM,48.0,41,83,14,30,21,36,11,29,27,19.0,9,5,4,31,29,117,-14.0,2018-19,1610612763,MEM,Memphis Grizzlies,2018-10-19,MEM vs. ATL,W,44,81,13,26,30,43,7,34,32,17.0,9,131,0,0,-14.0,0.0,NaN
9,2018-19,1610612737,ATL,Atlanta Hawks,21800036,2018-10-21,ATL @ CLE,48.0,48,98,22,47,15,19,9,35,32,12.0,9,7,2,21,25,133,22.0,2018-19,1610612739,CLE,Cleveland Cavaliers,2018-10-21,CLE vs. ATL,L,46,99,10,27,9,14,15,34,23,18.0,5,111,0,1,22.0,0.0,0.0
13,2018-19,1610612737,ATL,Atlanta Hawks,21800052,2018-10-24,ATL vs. DAL,48.0,37,91,15,38,22,31,10,44,28,15.0,6,6,9,26,27,111,7.0,2018-19,1610612742,DAL,Dallas Mavericks,2018-10-24,DAL @ ATL,L,38,91,12,40,16,26,9,43,25,17.0,7,104,1,1,7.0,1.0,0.0
17,2018-19,1610612737,ATL,Atlanta Hawks,21800076,2018-10-27,ATL vs. CHI,48.0,27,84,12,40,19,23,11,37,20,22.0,9,8,6,13,21,85,-12.0,2018-19,1610612741,CHI,Chicago Bulls,2018-10-27,CHI @ ATL,W,36,83,12,33,13,16,8,44,25,23.0,14,97,1,0,-12.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2266,2018-19,1610612764,WAS,Washington Wizards,21801148,2019-03-31,WAS @ DEN,48.0,37,92,6,24,15,20,9,45,24,12.0,9,6,5,21,18,95,5.0,2018-19,1610612743,DEN,Denver Nuggets,2019-03-31,DEN vs. WAS,L,35,93,7,36,13,20,12,45,18,11.0,6,90,0,1,5.0,0.0,1.0
1478,2018-19,1610612764,WAS,Washington Wizards,21801167,2019-04-03,WAS vs. CHI,48.0,45,97,5,27,19,26,9,30,24,12.0,8,9,5,18,23,114,-1.0,2018-19,1610612741,CHI,Chicago Bulls,2019-04-03,CHI @ WAS,W,48,90,5,19,14,17,9,40,22,20.0,7,115,1,0,-1.0,1.0,0.0
4886,2018-19,1610612764,WAS,Washington Wizards,21801182,2019-04-05,WAS vs. SAS,48.0,42,88,9,32,19,22,9,25,22,10.0,2,4,4,16,18,112,-17.0,2018-19,1610612759,SAS,San Antonio Spurs,2019-04-05,SAS @ WAS,W,51,91,10,25,17,20,12,30,34,9.0,3,129,1,0,-17.0,0.0,1.0
4414,2018-19,1610612764,WAS,Washington Wizards,21801204,2019-04-07,WAS @ NYK,48.0,43,103,10,34,14,18,21,29,24,9.0,12,4,9,16,14,110,-3.0,2018-19,1610612752,NYK,New York Knicks,2019-04-07,NYK vs. WAS,W,41,80,17,34,14,19,7,36,24,15.0,5,113,0,0,-3.0,0.0,0.0


In [32]:
# def get_cumulative_stats(df, n=5):
#     df = df.copy()
#     stats = ['MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M',
#        'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST',
#        'TOV', 'STL', 'BLK', 'BLKA', 'PF', 'PFD', 'PTS', 'PLUS_MINUS',
#        'FGM_opp', 'FGA_opp', 'FG_PCT_opp', 'FG3M_opp', 'FG3A_opp',
#        'FG3_PCT_opp', 'FTM_opp', 'FTA_opp', 'FT_PCT_opp', 'OREB_opp',
#        'DREB_opp', 'REB_opp', 'AST_opp', 'TOV_opp', 'STL_opp', 'BLK_opp',
#        'BLKA_opp', 'PF_opp', 'PFD_opp', 'PTS_opp', 'PLUS_MINUS_opp',
#        'WIN']
#     all_teams_cume_stats = []
    
#     team_abbreviations = df['TEAM_ABBREVIATION'].unique()
#     for team in team_abbreviations:
#         team_df = df.loc[df['TEAM_ABBREVIATION'] == team].sort_values(['GAME_DATE'])        
#         team_df[stats] = team_df[stats].shift(1).expanding(min_periods=5).mean()
#         last_n_games = df.loc[df['TEAM_ABBREVIATION'] == team].sort_values(['GAME_DATE'])    

#         last_n_games[stats] = last_n_games[stats].shift(1).rolling(n, min_periods=5).mean()
#         combined_df = pd.merge(team_df, last_n_games[stats], left_index=True, right_index=True, suffixes =['_avg', '_last{}'.format(n)])
#         combined_df.sort_values(['GAME_DATE'], inplace=True)
#         all_teams_cume_stats.append(combined_df)
        
#     full_df = pd.concat(all_teams_cume_stats)
    
#     return full_df


# full_gamelogs_cume = get_cumulative_stats(clean_gamelogs_2018)
# full_gamelogs_cume
        

KeyError: "['PLUS_MINUS_opp', 'FT_PCT', 'BLKA_opp', 'FG3_PCT_opp', 'FT_PCT_opp', 'REB_opp', 'REB', 'FG_PCT', 'FG3_PCT', 'PF_opp', 'PFD_opp', 'FG_PCT_opp', 'BLK_opp'] not in index"

In [72]:
def get_weighted_cume_stats(df, n=10):
    df = df.copy()
    stats = ['MIN', 'FGM', 'FGA', 'FG3M',
             'FG3A', 'FTM', 'FTA', 'OREB', 'DREB', 'AST',
             'TOV', 'STL', 'BLK', 'BLKA', 'PF', 'PFD', 'PTS',
             'PLUS_MINUS', 'FGM_opp', 'FGA_opp', 'FG3M_opp',
             'FG3A_opp', 'FTM_opp', 'FTA_opp', 'OREB_opp', 
             'DREB_opp', 'AST_opp', 'TOV_opp', 'STL_opp', 
             'PTS_opp', 'WIN']
    all_teams_cume_stats = []
    
    team_abbreviations = df['TEAM_ABBREVIATION'].unique()
    
    weights = np.array([0.05, 0.05, 0.05, 0.05, 0.05,
                       0.15, 0.15, 0.15, 0.15, 0.15])
    
    sum_weights = np.sum(weights)
    
    for team in team_abbreviations:
        team_df = df.loc[df['TEAM_ABBREVIATION'] == team].sort_values(['GAME_DATE'])        
        team_df[stats] = (team_df[stats].shift(1).rolling(window=n)
                         .apply(lambda row: np.sum(weights*row) / sum_weights, raw=False))
                        
        
        
        team_df.sort_values(['GAME_DATE'], inplace=True)
        all_teams_cume_stats.append(team_df)
        
    full_df = pd.concat(all_teams_cume_stats)
    
    return full_df

weighted_stats_2018 = get_weighted_cume_stats(clean_gamelogs_2018)
weighted_stats_2018

,SEASON_YEAR,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,MIN,FGM,FGA,FG3M,FG3A,FTM,FTA,OREB,DREB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,SEASON_YEAR_opp,TEAM_ID_opp,TEAM_ABBREVIATION_opp,TEAM_NAME_opp,GAME_DATE_opp,MATCHUP_opp,WL_opp,FGM_opp,FGA_opp,FG3M_opp,FG3A_opp,FTM_opp,FTA_opp,OREB_opp,DREB_opp,AST_opp,TOV_opp,STL_opp,PTS_opp,HOME_GAME,WIN,point_diff,prev_WIN,prev2_WIN
1,2018-19,1610612737,ATL,Atlanta Hawks,21800007,2018-10-17,ATL @ NYK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-19,1610612752,NYK,New York Knicks,2018-10-17,NYK vs. ATL,W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,-19.0,NaN,NaN
5,2018-19,1610612737,ATL,Atlanta Hawks,21800020,2018-10-19,ATL @ MEM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-19,1610612763,MEM,Memphis Grizzlies,2018-10-19,MEM vs. ATL,W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,-14.0,0.0,NaN
9,2018-19,1610612737,ATL,Atlanta Hawks,21800036,2018-10-21,ATL @ CLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-19,1610612739,CLE,Cleveland Cavaliers,2018-10-21,CLE vs. ATL,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,22.0,0.0,0.0
13,2018-19,1610612737,ATL,Atlanta Hawks,21800052,2018-10-24,ATL vs. DAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-19,1610612742,DAL,Dallas Mavericks,2018-10-24,DAL @ ATL,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,7.0,1.0,0.0
17,2018-19,1610612737,ATL,Atlanta Hawks,21800076,2018-10-27,ATL vs. CHI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-19,1610612741,CHI,Chicago Bulls,2018-10-27,CHI @ ATL,W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,-12.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2266,2018-19,1610612764,WAS,Washington Wizards,21801148,2019-03-31,WAS @ DEN,48.25,41.75,89.40,10.05,31.05,19.95,26.35,11.95,32.65,23.95,13.15,7.15,3.35,6.10,18.20,21.90,113.50,-5.15,2018-19,1610612743,DEN,Denver Nuggets,2019-03-31,DEN vs. WAS,L,45.75,89.60,12.95,33.50,14.20,18.05,10.75,34.60,28.75,14.00,7.80,118.65,0,0.25,5.0,0.0,1.0
1478,2018-19,1610612764,WAS,Washington Wizards,21801167,2019-04-03,WAS vs. CHI,48.25,41.20,90.80,9.75,30.50,19.80,26.65,12.20,34.40,24.05,13.30,7.85,3.95,6.30,19.20,21.85,111.95,-4.40,2018-19,1610612741,CHI,Chicago Bulls,2019-04-03,CHI @ WAS,W,44.65,90.35,12.00,34.25,15.05,19.50,11.05,36.45,27.15,14.30,7.65,116.35,1,0.35,-1.0,1.0,0.0
4886,2018-19,1610612764,WAS,Washington Wizards,21801182,2019-04-05,WAS vs. SAS,48.25,41.80,91.55,8.95,29.85,20.20,27.30,11.05,33.85,23.90,13.05,8.15,4.85,5.70,19.20,22.15,112.75,-3.75,2018-19,1610612759,SAS,San Antonio Spurs,2019-04-05,SAS @ WAS,W,45.15,90.00,10.85,31.65,15.35,19.55,10.45,37.70,26.65,15.50,7.65,116.50,1,0.35,-17.0,0.0,1.0
4414,2018-19,1610612764,WAS,Washington Wizards,21801204,2019-04-07,WAS @ NYK,48.25,42.15,91.70,8.85,29.70,19.05,25.75,10.85,33.00,23.80,12.20,7.65,4.70,5.65,18.55,21.40,112.20,-4.85,2018-19,1610612752,NYK,New York Knicks,2019-04-07,NYK vs. WAS,W,45.60,90.25,10.65,30.45,15.20,18.95,10.60,36.65,26.30,14.45,6.95,117.05,0,0.30,-3.0,0.0,0.0


In [35]:
weighted_stats_2018.loc[weighted_stats_2018['TEAM_ABBREVIATION'] == 'ATL'].head(15)

,SEASON_YEAR,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,MIN,FGM,FGA,FG3M,FG3A,FTM,FTA,OREB,DREB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,SEASON_YEAR_opp,TEAM_ID_opp,TEAM_ABBREVIATION_opp,TEAM_NAME_opp,GAME_DATE_opp,MATCHUP_opp,WL_opp,FGM_opp,FGA_opp,FG3M_opp,FG3A_opp,FTM_opp,FTA_opp,OREB_opp,DREB_opp,AST_opp,TOV_opp,STL_opp,PTS_opp,HOME_GAME,WIN,point_diff,prev_WIN,prev2_WIN
1,2018-19,1610612737,ATL,Atlanta Hawks,21800007,2018-10-17,ATL @ NYK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-19,1610612752,NYK,New York Knicks,2018-10-17,NYK vs. ATL,W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,-19.0,NaN,NaN
5,2018-19,1610612737,ATL,Atlanta Hawks,21800020,2018-10-19,ATL @ MEM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-19,1610612763,MEM,Memphis Grizzlies,2018-10-19,MEM vs. ATL,W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,-14.0,0.0,NaN
9,2018-19,1610612737,ATL,Atlanta Hawks,21800036,2018-10-21,ATL @ CLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-19,1610612739,CLE,Cleveland Cavaliers,2018-10-21,CLE vs. ATL,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,22.0,0.0,0.0
13,2018-19,1610612737,ATL,Atlanta Hawks,21800052,2018-10-24,ATL vs. DAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-19,1610612742,DAL,Dallas Mavericks,2018-10-24,DAL @ ATL,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,7.0,1.0,0.0
17,2018-19,1610612737,ATL,Atlanta Hawks,21800076,2018-10-27,ATL vs. CHI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-19,1610612741,CHI,Chicago Bulls,2018-10-27,CHI @ ATL,W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,-12.0,1.0,1.0
21,2018-19,1610612737,ATL,Atlanta Hawks,21800088,2018-10-29,ATL @ PHI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-19,1610612755,PHI,Philadelphia 76ers,2018-10-29,PHI vs. ATL,W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,-21.0,0.0,1.0
25,2018-19,1610612737,ATL,Atlanta Hawks,21800097,2018-10-30,ATL @ CLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-19,1610612739,CLE,Cleveland Cavaliers,2018-10-30,CLE vs. ATL,W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,-22.0,0.0,0.0
29,2018-19,1610612737,ATL,Atlanta Hawks,21800114,2018-11-01,ATL vs. SAC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-19,1610612758,SAC,Sacramento Kings,2018-11-01,SAC @ ATL,W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,-31.0,0.0,0.0
33,2018-19,1610612737,ATL,Atlanta Hawks,21800128,2018-11-03,ATL vs. MIA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-19,1610612748,MIA,Miami Heat,2018-11-03,MIA @ ATL,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,5.0,0.0,0.0
37,2018-19,1610612737,ATL,Atlanta Hawks,21800149,2018-11-06,ATL @ CHA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-19,1610612766,CHA,Charlotte Hornets,2018-11-06,CHA vs. ATL,W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,-11.0,1.0,0.0


In [36]:
def final_preparation(df):
       
    # Merge the data once more, since we have team A's data, Team A's opponent's data, Team B's data ,Team B's opponent's data  
    final_merged_df = pd.merge(df, df, on='GAME_ID')

    # Filter out non-sense rows where the same team is playing itself
    final_merged_df = final_merged_df.loc[final_merged_df['TEAM_ABBREVIATION_x'] != final_merged_df['TEAM_ABBREVIATION_y']]

    # Only take the home games (This will include the entire season, as one team is always at home)
    # It just makes it so the home team is always the first set of stats
    final_merged_df = final_merged_df.loc[final_merged_df['HOME_GAME_x'] == 1]

    # Resort the data in chronological order
    final_merged_df = final_merged_df.sort_values(['GAME_DATE_x'])

    # For readability, rename columns    
    new_col_names = ['HOME_' + x[:-2] if x[-2:] == '_x' else 'AWAY_' + x[:-2] for x in final_merged_df.columns.to_list()]
    final_merged_df.columns = new_col_names
    
    final_merged_df.rename(columns={'HOME_point_diff':'point_diff'}, inplace=True)
    
    # Drop unnecessary columns
    cols_to_drop = ['HOME_TEAM_ID', 'HOME_TEAM_NAME', 'HOME_MATCHUP',
                    'HOME_SEASON_YEAR_opp', 'HOME_GAME_DATE',
                    'HOME_TEAM_ID_opp', 'HOME_TEAM_ABBREVIATION_opp', 
                    'HOME_TEAM_NAME_opp', 'HOME_GAME_DATE_opp',
                    'HOME_MATCHUP_opp', 'HOME_WL_opp',
                    'HOME_HOME_GAME', 'AWAY_GAME_', 'AWAY_SEASON_YEAR',
                    'AWAY_TEAM_ID', 'AWAY_MATCHUP', 'AWAY_TEAM_NAME',
                    'AWAY_GAME_DATE', 'AWAY_HOME_GAME', 'AWAY_SEASON_YEAR_opp',
                   'AWAY_TEAM_ID_opp', 'AWAY_TEAM_ABBREVIATION_opp',
                   'AWAY_TEAM_NAME_opp', 'AWAY_GAME_DATE_opp',
                   'AWAY_MATCHUP_opp', 'AWAY_WL_opp', 'AWAY_point_diff']
    final_merged_df = final_merged_df.drop(columns=cols_to_drop)
    final_merged_df.dropna(inplace=True)
    return final_merged_df

final_merged_df_2018 = final_preparation(df=weighted_stats_2018)
final_merged_df_2018
                                                  
                               
                                                  
                                                

,HOME_SEASON_YEAR,HOME_TEAM_ABBREVIATION,HOME_MIN,HOME_FGM,HOME_FGA,HOME_FG3M,HOME_FG3A,HOME_FTM,HOME_FTA,HOME_OREB,HOME_DREB,HOME_AST,HOME_TOV,HOME_STL,HOME_BLK,HOME_BLKA,HOME_PF,HOME_PFD,HOME_PTS,HOME_PLUS_MINUS,HOME_FGM_opp,HOME_FGA_opp,HOME_FG3M_opp,HOME_FG3A_opp,HOME_FTM_opp,HOME_FTA_opp,HOME_OREB_opp,HOME_DREB_opp,HOME_AST_opp,HOME_TOV_opp,HOME_STL_opp,HOME_PTS_opp,HOME_WIN,point_diff,HOME_prev_WIN,HOME_prev2_WIN,AWAY_TEAM_ABBREVIATION,AWAY_MIN,AWAY_FGM,AWAY_FGA,AWAY_FG3M,AWAY_FG3A,AWAY_FTM,AWAY_FTA,AWAY_OREB,AWAY_DREB,AWAY_AST,AWAY_TOV,AWAY_STL,AWAY_BLK,AWAY_BLKA,AWAY_PF,AWAY_PFD,AWAY_PTS,AWAY_PLUS_MINUS,AWAY_FGM_opp,AWAY_FGA_opp,AWAY_FG3M_opp,AWAY_FG3A_opp,AWAY_FTM_opp,AWAY_FTA_opp,AWAY_OREB_opp,AWAY_DREB_opp,AWAY_AST_opp,AWAY_TOV_opp,AWAY_STL_opp,AWAY_PTS_opp,AWAY_WIN,AWAY_prev_WIN,AWAY_prev2_WIN
1250,2018-19,NYK,48.00,39.75,89.45,9.40,29.35,15.40,19.75,11.75,32.05,20.20,15.50,7.25,3.70,6.65,22.65,20.60,104.30,-4.70,40.15,83.65,10.15,29.10,18.55,25.10,10.70,33.15,22.75,15.20,8.20,109.00,0.30,-1.0,0.0,1.0,CHI,48.75,37.95,84.80,11.50,30.05,16.95,21.35,8.05,33.20,23.75,15.40,7.35,5.25,7.15,20.45,18.55,104.35,-7.50,41.15,90.10,12.45,35.80,17.10,21.65,11.40,36.05,24.30,14.30,9.00,111.85,0.15,0.0,0.0
3005,2018-19,IND,48.00,39.55,83.25,8.35,24.50,16.60,23.00,7.85,31.30,21.45,13.35,9.10,5.00,6.00,17.20,20.85,104.05,1.10,37.80,83.60,13.00,35.30,14.35,16.70,9.70,32.85,23.40,16.15,6.15,102.95,0.60,-6.0,0.0,1.0,PHI,48.25,37.20,84.25,10.60,31.20,26.20,32.75,10.45,38.60,24.50,18.50,6.95,5.10,5.15,24.70,25.95,111.20,-1.35,41.40,94.40,9.30,30.35,20.45,26.55,10.05,33.50,22.70,13.40,8.00,112.55,0.60,0.0,1.0
1254,2018-19,NOP,48.00,41.90,88.55,9.15,27.75,22.30,29.70,10.85,34.55,25.35,16.30,6.95,5.15,4.45,24.95,23.75,115.25,-6.45,44.50,91.70,12.10,30.65,20.60,26.55,9.00,34.40,27.80,13.30,9.15,121.70,0.20,9.0,0.0,0.0,CHI,50.50,39.15,86.35,10.65,29.75,18.15,23.40,8.90,34.30,22.20,16.25,7.70,5.90,6.75,20.85,20.30,107.10,-4.45,40.50,92.90,11.55,34.20,19.00,23.25,13.15,36.40,23.05,14.90,8.40,111.55,0.35,1.0,0.0
3405,2018-19,LAL,48.25,44.60,90.70,9.35,26.45,17.10,23.55,9.10,34.30,24.35,16.55,8.75,6.30,5.55,22.50,21.65,115.65,-2.20,45.15,95.05,9.85,30.30,17.70,22.25,13.00,34.65,24.90,16.05,8.10,117.85,0.45,4.0,0.0,1.0,MIN,48.00,39.05,93.30,9.90,30.90,19.15,24.20,11.10,30.35,21.85,11.60,9.35,6.75,5.60,20.10,22.40,107.15,-11.20,44.05,91.60,12.40,32.95,17.85,22.65,13.90,38.25,25.95,15.45,6.00,118.35,0.30,0.0,0.0
1778,2018-19,UTA,48.00,38.65,83.80,10.40,31.65,21.20,30.35,10.50,31.70,24.65,14.35,8.25,5.35,3.45,22.25,26.50,108.90,-3.25,42.65,85.75,10.75,28.70,16.10,22.35,8.45,33.50,22.85,14.35,8.35,112.15,0.30,15.0,0.0,0.0,DAL,48.50,40.40,87.35,12.40,33.20,18.30,23.50,10.50,33.75,23.35,17.90,6.05,4.50,4.75,22.25,24.60,111.50,-1.35,41.85,85.60,10.80,27.20,18.35,23.20,7.65,32.25,25.45,14.75,10.20,112.85,0.30,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2289,2018-19,DEN,48.00,40.20,89.50,9.60,29.55,15.30,20.95,9.40,32.05,25.40,11.70,8.80,3.65,5.20,21.35,19.35,105.30,-3.30,40.95,86.75,9.35,28.80,17.35,22.75,11.90,36.40,23.90,14.35,6.90,108.60,0.35,4.0,0.0,0.0,MIN,48.25,42.00,89.05,10.75,29.60,17.90,22.55,7.50,31.85,25.85,12.90,8.50,5.10,5.15,19.55,21.85,112.65,-6.50,45.40,91.45,13.15,34.30,15.20,19.90,11.35,37.35,26.60,15.15,7.30,119.15,0.35,0.0,0.0
4025,2018-19,MIL,48.50,44.10,92.80,13.35,38.05,21.90,28.15,9.10,43.25,25.55,13.20,7.60,6.25,4.50,19.85,21.90,123.45,6.65,42.65,97.85,16.90,43.40,14.60,20.80,10.25,37.40,26.15,12.45,5.55,116.80,0.70,-11.0,1.0,0.0,OKC,48.25,43.40,96.15,13.20,36.95,15.35,21.10,15.60,33.35,24.55,12.70,7.50,4.35,4.90,21.20,22.05,115.35,3.05,40.45,85.40,14.50,38.05,16.90,21.10,7.30,33.70,23.70,14.55,7.35,112.30,0.70,1.0,1.0
2754,2018-19,MEM,49.00,38.30,87.20,12.25,33.40,19.40,24.35,10.30,35

In [37]:
corr = final_merged_df_2018.select_dtypes('number').corr()

correlated_features = []
for i in range(len(corr.columns)):
    for j in range(i):
        if abs(corr.iloc[i, j]) >= 0.9:
            correlated_features.append(corr.columns[i])
            correlated_features.append(corr.columns[j])
print("Correlated Features:", list(correlated_features))
print("Num Features Removed ", len(correlated_features))

Correlated Features: ['HOME_FTA', 'HOME_FTM', 'HOME_FTA_opp', 'HOME_FTM_opp', 'AWAY_FTA', 'AWAY_FTM', 'AWAY_FTA_opp', 'AWAY_FTM_opp']
Num Features Removed  8


## Feature Selection

In [ ]:
import seaborn as sns
from sklearn.feature_selection import VarianceThreshold

corr = final_merged_df_2018.drop(columns=['HOME_SEASON_YEAR', 'HOME_TEAM_ABBREVIATION', 'AWAY_TEAM_ABBREVIATION']).corr()

mask = np.triu(np.ones_like(corr, dtype=bool))

fig, ax = plt.subplots(figsize=(50,50))

sns.heatmap(corr, mask=mask, square=True, linewidths=0.5)

plt.savefig('correlations.jpg')


In [ ]:
def remove_correlated_features(final_merged_df, threshold=1):
    """Removes correlated features from dataset
    INPUTS:
    final_merged_df
    threshold: float between 0 and 1 indicating how highly correlated features
    must be to be removed
    OUTPUT: final_merged_df with the correlated feature columns removed
    """
    corr = final_merged_df.select_dtypes('number').corr()

    correlated_features = set()
    for i in range(len(corr.columns)):
        for j in range(i):
            if abs(corr.iloc[i, j]) >= threshold:
                colname = corr.columns[i]
                correlated_features.add(colname)
    print("Correlated Features:", list(correlated_features))
    print("Num Features Removed ", len(correlated_features))
    final_merged_df.drop(columns=list(correlated_features), inplace=True)
    return final_merged_df

In [ ]:
# Remove perfectly correlated features
final_merged_df = remove_correlated_features(final_merged_df_2018)
final_merged_df

In [ ]:
seasons = ['20{}-{}'.format(x, x+1) for x in range(14, 21)]

gamelogs_2014_2020 = []

for season in seasons:
    df = pd.read_csv('./data/gamelog_{}.csv'.format(season))
    df = clean_team_gamelog(df)
    df = get_cumulative_stats(df)
    df = final_preparation(df)
#     df = remove_correlated_features(df)
    gamelogs_2010_2020.append(df)
    
gamelogs_all_seasons = pd.concat(gamelogs_2010_2020)
    

In [ ]:
gamelogs_all_seasons.to_csv('./data/gamelogs_all_seasons.csv', index=False)

In [ ]:
gamelogs_all_seasons = pd.read_csv('./data/gamelogs_all_seasons.csv')
gamelogs_all_seasons

In [ ]:
seasons = ['2014-15', '2015-16', '2016-17', '2017-18', '2018-19', '2019-20']
gamelogs_2015_20 = gamelogs_all_seasons.loc[gamelogs_all_seasons['HOME_SEASON_YEAR'].isin(seasons)]

In [ ]:
gamelogs_all_seasons = gamelogs_all_seasons.dropna(axis=1)

X_train = gamelogs_all_seasons.loc[gamelogs_all_seasons['HOME_GAME_DATE'] <= '2018-04-10']
X_train.drop(columns = ['HOME_SEASON_YEAR', 'HOME_TEAM_ABBREVIATION', 'HOME_GAME_DATE',
                       'AWAY_TEAM_ABBREVIATION', 'AWAY_SEASON_YEAR_opp', 'AWAY_TEAM_ID_opp',
                       'AWAY_TEAM_ABBREVIATION_opp', 'AWAY_GAME_DATE_opp', 'AWAY_MATCHUP_opp',
                       'AWAY_WL_opp', 'HOME_SEASON_YEAR_opp', 'HOME_TEAM_ID_opp', 
                        'HOME_TEAM_ABBREVIATION_opp', 'HOME_TEAM_NAME_opp', 'HOME_GAME_DATE_opp', 
                        'HOME_MATCHUP_opp', 'HOME_WL_opp', 'AWAY_TEAM_NAME_opp', 'point_diff'],inplace=True)


y_train = gamelogs_all_seasons.loc[gamelogs_all_seasons['HOME_GAME_DATE'] <= '2018-04-10', 'point_diff']


X_test = gamelogs_all_seasons.loc[gamelogs_all_seasons['HOME_GAME_DATE'] > '2018-04-10']
X_test.drop(columns = ['HOME_SEASON_YEAR', 'HOME_TEAM_ABBREVIATION', 'HOME_GAME_DATE',
                       'AWAY_TEAM_ABBREVIATION', 'AWAY_SEASON_YEAR_opp', 'AWAY_TEAM_ID_opp',
                       'AWAY_TEAM_ABBREVIATION_opp', 'AWAY_GAME_DATE_opp', 'AWAY_MATCHUP_opp',
                       'AWAY_WL_opp', 'HOME_SEASON_YEAR_opp', 'HOME_TEAM_ID_opp', 
                        'HOME_TEAM_ABBREVIATION_opp', 'HOME_TEAM_NAME_opp', 'HOME_GAME_DATE_opp', 
                        'HOME_MATCHUP_opp', 'HOME_WL_opp', 'AWAY_TEAM_NAME_opp', 'point_diff'],inplace=True)


y_test = gamelogs_all_seasons.loc[gamelogs_all_seasons['HOME_GAME_DATE'] > '2018-04-10', 'point_diff'] 



    
    

In [ ]:
print(X_train.shape, y_train.shape)

In [ ]:
pd.options.display.max_columns=170

In [ ]:
X_train

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, ElasticNet, Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import learning_curve
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import numpy as np
from sklearn.svm import SVR

In [ ]:
model = LinearRegression()
pipeline = Pipeline([('normalize', Normalizer()),
                    ('est', model)
                    ]
                   )


train_sizes = [1, 100, 500, 2000, 5000, 7161]

train_sizes, train_scores, validation_scores = learning_curve(estimator = pipeline,
                                                             X = X_train,
                                                             y = y_train,
                                                             train_sizes = train_sizes,
                                                             scoring='neg_mean_absolute_error')

plt.style.use('fivethirtyeight')

plt.plot(train_sizes, train_scores, label='Training Error')
plt.plot(train_sizes, validation_scores, label='Validation Error')
plt.ylabel('Mean Abs Error', fontsize=14)
plt.xlabel('Training Set Size', fontsize=14)
plt.title('Learning Curve')
plt.legend()
plt.show()

In [ ]:
model = LinearRegression()
pipeline = Pipeline([('normalize', Normalizer()),
                    ('est', model)
                    ]
                   )
pipeline.fit(X_train, y_train)

y_preds = pipeline.predict(X_test)

mae = mean_absolute_error(y_test, y_preds)
print("{} Mean Absolute Error:".format(str(model)), mae)

In [ ]:
coefficients = pipeline.named_steps['est'].coef_

sorted(list(zip(X_train.columns, coefficients)), key=lambda x: x[1], reverse=True)



In [ ]:
pipeline = Pipeline([('normalize', Normalizer()),
                    ('est', Ridge())
                    ]
                   )

params = {'est__alpha' : [0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1, 3, 10]}

gs_ridge = GridSearchCV(pipeline, param_grid = params)

gs_ridge.fit(X_train, y_train)

y_preds = gs_ridge.predict(X_test)

mae = mean_absolute_error(y_test, y_preds)
print("Ridge Regression: Mean Absolute Error:", mae)

In [ ]:
best_ridge = gs_ridge.best_estimator_
best_ridge

ridge = Ridge(alpha=0.1)
ridge.fit(X_train, y_train)
y_preds = ridge.predict(X_test)

mae = mean_absolute_error(y_test, y_preds)
print("Ridge Regression: Mean Absolute Error:", mae)

coefficients = ridge.coef_

sorted(list(zip(X_train.columns, coefficients)), key=lambda x: x[1], reverse=True)

In [ ]:
pipeline = Pipeline([('normalize', Normalizer()),
                    ('est', Lasso())
                    ]
                   )

params = {'est__alpha' : [0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1, 3, 10]}

gs_lasso = GridSearchCV(pipeline, param_grid = params)

gs_lasso.fit(X_train, y_train)

y_preds = gs_lasso.predict(X_test)

mae = mean_absolute_error(y_test, y_preds)
print("Lasso Regression: Mean Absolute Error:", mae)

In [ ]:
gs_lasso.best_params_

In [ ]:
lasso = Lasso(alpha=0.001)
lasso.fit(X_train, y_train)

coefficients = lasso.coef_

sorted(list(zip(X_train.columns, coefficients)), key=lambda x: x[1])

In [ ]:
from xgboost import XGBRegressor
from xgboost import plot_importance

In [ ]:
xgbr = XGBRegressor()

xgbr.fit(X_train, y_train)

y_preds = xgbr.predict(X_test)

mae = mean_absolute_error(y_test, y_preds)

print("XGB Mean Absolute Error:", mae)

In [ ]:
params = {'eta':np.linspace(0.1, 0.3, 3),
         'min_child_weight':[1,3, 5],
         'max_depth':[3, 6, 9],
         'gamma':[0, 0.01, 0.1],
         'subsample':[0.5, 0.7, 1],
         'colsample_bytree':[0.5, 0.7, 1]
         }

xgb_gs = GridSearchCV(estimator=XGBRegressor(),
                     param_grid=params,
                      scoring='neg_mean_absolute_error'
                     )

xgb_gs.fit(X_train, y_train)

y_preds = xgb_gs.predict(X_test)

mae = mean_absolute_error(y_test, y_preds)

print("XGB Mean Absolute Error:", mae)

In [ ]:
best_xgb = xgb_gs.best_estimator_

y_preds = xgb_gs.predict(X_test)

mae = mean_absolute_error(y_test, y_preds)

print("XGB Mean Absolute Error:", mae)

In [ ]:
print(xgb_gs.best_params_)

In [ ]:
sorted(list(zip(X_train.columns, best_xgb.feature_importances_)), key=lambda x: x[1], reverse=True)

In [ ]:
fig, ax = plt.subplots(figsize=(8,20))
plot_importance(best_xgb, ax=ax)

In [ ]:
rf = RandomForestRegressor()

rf.fit(X_train, y_train)

y_preds = rf.predict(X_test)

mae = mean_absolute_error(y_test, y_preds)

print("Random Forest Mean Absolute Error", mae)

In [ ]:
best_xgb.feature_importances_

## Model with Weighted Stats (see get_weighted_cume_stats function)

In [38]:
seasons = ['20{}-{}'.format(x, x+1) for x in range(14, 21)]

gamelogs_2014_2020 = []

for season in seasons:
    df = pd.read_csv('./data/gamelog_{}.csv'.format(season))
    df = clean_team_gamelog(df)
    df = get_weighted_cume_stats(df, n=10)
    df = final_preparation(df)
#     df = remove_correlated_features(df, 0.8)
    gamelogs_2014_2020.append(df)
    
gamelogs_2014_20_weighted = pd.concat(gamelogs_2014_2020)
    

In [42]:
gamelogs_2014_20_weighted.to_csv('./data/gamelogs_all_seasons_weighted.csv')

In [43]:
gamelogs_2014_20_weighted

,HOME_SEASON_YEAR,HOME_TEAM_ABBREVIATION,HOME_MIN,HOME_FGM,HOME_FGA,HOME_FG3M,HOME_FG3A,HOME_FTM,HOME_FTA,HOME_OREB,HOME_DREB,HOME_AST,HOME_TOV,HOME_STL,HOME_BLK,HOME_BLKA,HOME_PF,HOME_PFD,HOME_PTS,HOME_PLUS_MINUS,HOME_FGM_opp,HOME_FGA_opp,HOME_FG3M_opp,HOME_FG3A_opp,HOME_FTM_opp,HOME_FTA_opp,HOME_OREB_opp,HOME_DREB_opp,HOME_AST_opp,HOME_TOV_opp,HOME_STL_opp,HOME_PTS_opp,HOME_WIN,point_diff,HOME_prev_WIN,HOME_prev2_WIN,AWAY_TEAM_ABBREVIATION,AWAY_MIN,AWAY_FGM,AWAY_FGA,AWAY_FG3M,AWAY_FG3A,AWAY_FTM,AWAY_FTA,AWAY_OREB,AWAY_DREB,AWAY_AST,AWAY_TOV,AWAY_STL,AWAY_BLK,AWAY_BLKA,AWAY_PF,AWAY_PFD,AWAY_PTS,AWAY_PLUS_MINUS,AWAY_FGM_opp,AWAY_FGA_opp,AWAY_FG3M_opp,AWAY_FG3A_opp,AWAY_FTM_opp,AWAY_FTA_opp,AWAY_OREB_opp,AWAY_DREB_opp,AWAY_AST_opp,AWAY_TOV_opp,AWAY_STL_opp,AWAY_PTS_opp,AWAY_WIN,AWAY_prev_WIN,AWAY_prev2_WIN
2794,2014-15,MEM,48.00,35.60,80.60,5.30,13.85,22.00,27.25,11.75,29.05,20.25,11.85,8.15,3.05,4.45,19.75,21.90,98.50,4.80,35.80,80.50,7.70,20.40,14.40,19.65,11.55,30.85,16.90,15.65,6.40,93.70,0.90,26.0,1.0,1.0,HOU,48.00,32.05,78.70,10.80,33.10,19.35,29.20,13.85,31.70,18.00,18.35,9.25,6.15,6.80,23.85,23.85,94.25,7.75,31.40,79.45,5.75,20.10,17.95,23.95,11.85,30.35,18.00,18.20,8.45,86.50,0.90,1.0,1.0
977,2014-15,CHA,49.25,36.15,83.65,6.00,17.05,18.60,25.25,10.30,32.70,19.60,13.45,5.55,3.70,3.80,17.80,24.30,96.90,-5.20,39.30,85.40,8.05,23.45,15.45,18.85,9.75,33.70,24.15,13.30,6.30,102.10,0.40,-27.0,0.0,1.0,DAL,48.00,42.00,85.60,8.85,26.25,18.85,23.05,10.70,29.45,25.05,11.05,8.20,6.50,3.30,22.15,24.55,111.70,12.60,35.75,78.85,9.10,24.05,18.50,25.25,12.60,30.85,21.10,20.45,5.00,99.10,0.75,1.0,1.0
2321,2014-15,DET,48.75,35.55,83.60,8.65,23.50,14.10,21.35,11.40,34.35,17.80,13.30,5.30,5.00,5.30,21.50,20.35,93.85,-3.10,36.30,84.65,6.15,19.55,18.20,24.35,11.80,33.95,21.80,11.05,7.20,96.95,0.30,-14.0,0.0,1.0,ORL,48.50,37.60,79.25,7.40,17.25,15.20,21.25,8.20,32.90,20.10,16.00,5.85,3.15,5.65,21.90,21.10,97.80,0.10,37.85,82.75,5.10,17.30,16.90,21.40,8.85,31.60,21.90,13.10,6.90,97.70,0.45,0.0,1.0
4430,2014-15,UTA,48.00,35.50,76.05,7.25,21.80,17.85,23.90,9.65,29.20,22.60,14.70,6.45,5.20,3.65,19.15,19.60,96.10,-6.80,39.70,83.85,7.35,19.15,16.15,20.45,11.55,29.25,18.30,11.00,7.80,102.90,0.35,17.0,0.0,1.0,OKC,48.75,32.45,79.30,7.60,23.35,15.25,21.05,10.05,33.05,18.60,15.50,5.10,6.40,4.60,22.15,21.05,87.75,-1.45,33.40,82.60,7.85,27.80,14.55,21.35,13.05,31.60,19.50,13.30,7.10,89.20,0.30,0.0,0.0
3913,2014-15,MIL,48.25,35.55,80.90,4.65,18.05,14.50,20.10,10.30,32.40,20.05,16.65,8.95,4.45,4.80,23.65,19.35,90.25,-0.80,32.80,79.55,7.55,20.95,17.90,25.40,10.45,32.05,19.50,16.45,8.30,91.05,0.55,4.0,1.0,0.0,NYK,48.00,38.15,82.00,6.90,17.20,14.30,18.10,10.35,29.40,19.80,13.30,6.30,3.35,3.35,25.15,19.35,97.50,-0.30,33.35,75.35,10.00,23.20,21.10,27.80,9.45,30.20,20.60,14.90,6.85,97.80,0.30,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
650,2020-21,SAC,48.00,42.25,86.00,12.70,32.25,17.95,25.05,8.70,30.85,27.25,13.80,5.90,4.50,4.80,17.85,21.45,115.15,-13.65,47.45,91.50,17.10,39.45,16.80,20.85,10.10,33.25,27.65,12.20,9.10,128.80,0.30,-5.0,0.0,0.0,NOP,48.50,39.90,86.25,9.80,30.35,16.15,22.25,11.45,36.25,22.85,16.00,6.10,3.95,5.35,18.50,20.30,105.75,-5.35,38.65,86.25,16.35,41.55,17.45,21.85,8.70,32.50,28.30,12.30,8.35,111.10,0.20,0.0,0.0
230,2020-21,DAL,48.50,40.35,88.20,14.70,39.80,14.85,20.95,9.40,37.55,23.20,13.00,7.25,6.30,3.35,21.15,19.45,110.25,5.10,38.65,89.90,11.35,36.80,16.50,23.60,11.60,36.20,21.45,13.55,7.60,105.15,0.60,-16.0,0.0,1.0,CHI,49.00,43.85,87.90,15.30,36.80,17.20,21.35,8.15,38.05,26.05,18.70,6.80,4.05,4.35,22.30,20.45,120.20,-1.25,43.15,92.20,15.40,39.95,19.75,26.05,9.60,31.80,25.35,12.85,9.35,121.45,0.30,0.0,0.0
137,2020-21,BOS,48.00,42.65,88.15,12.90,33.30,16.35,20.90,10.75,36.25,23.45,15.45,8.30,5.95,4

In [46]:
training_seasons = ['2014-15', '2015-16', '2016-17', '2017-18', '2018-19']
test_season = ['2019-20', '2020-21']

X_train = gamelogs_2014_20_weighted.loc[gamelogs_2014_20_weighted['HOME_SEASON_YEAR'].isin(training_seasons)]
X_train.drop(columns = ['HOME_SEASON_YEAR', 'HOME_TEAM_ABBREVIATION',
                       'AWAY_TEAM_ABBREVIATION', 'point_diff'], inplace=True)

y_train = gamelogs_2014_20_weighted.loc[gamelogs_2014_20_weighted['HOME_SEASON_YEAR'].isin(training_seasons), 'point_diff']


X_test = gamelogs_2014_20_weighted.loc[gamelogs_2014_20_weighted['HOME_SEASON_YEAR'].isin(test_season)]
X_test.drop(columns = ['HOME_SEASON_YEAR', 'HOME_TEAM_ABBREVIATION',
                       'AWAY_TEAM_ABBREVIATION', 'point_diff'],inplace=True)


y_test = gamelogs_2014_20_weighted.loc[gamelogs_2014_20_weighted['HOME_SEASON_YEAR'].isin(test_season), 'point_diff']



    

## Training Models with Weighted Cumulative Stats

In [ ]:
X_train

In [47]:
from sklearn.linear_model import ElasticNet, Lasso, Ridge
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import Normalizer
from sklearn.metrics import mean_absolute_error

In [ ]:
pipeline = Pipeline([('normalize', Normalizer()),
                    ('est', Lasso())
                    ]
                   )

params = {'est__alpha' : [0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1, 3, 10]}

gs_lasso = GridSearchCV(pipeline, param_grid = params)

gs_lasso.fit(X_train, y_train)

best_lasso = gs_lasso.best_estimator_
print('parameters:', gs_lasso.best_params_)

y_preds = best_lasso.predict(X_test)

mae = mean_absolute_error(y_test, y_preds)
print("Lasso Regression: Mean Absolute Error:", mae)


In [ ]:
coefficients = best_lasso.named_steps['est'].coef_

sorted(list(zip(X_train.columns, coefficients)), key = lambda x: x[1], reverse=True)

In [ ]:
en = ElasticNet()

pipeline = Pipeline([('normalize', Normalizer()),
                    ('est', ElasticNet())
                    ]
                   )

params = {'est__alpha' : [0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1, 3, 10],
         'est__l1_ratio':np.linspace(0.1, 0.9, 10)}

gs_en = GridSearchCV(pipeline, param_grid = params, scoring = 'neg_mean_absolute_error')

gs_en.fit(X_train, y_train)

In [ ]:
best_en = gs_en.best_estimator_
print(gs_en.best_params_)

y_preds = best_en.predict(X_test)
mae = mean_absolute_error(y_test, y_preds)
print("Elastic Net MAE:", mae)

In [48]:
pipeline = Pipeline([('normalize', Normalizer()),
                    ('est', SVR())
                    ]
                   )

params = {'est__kernel':['rbf', 'linear', 'poly'],
         'est__degree':[3,4,5],
         'est__gamma':['scale', 'auto'],
         'est__C':np.logspace(-2, 2, 5)}

gs_svr = GridSearchCV(pipeline, param_grid=params, scoring='neg_mean_absolute_error')

gs_svr.fit(X_train, y_train)



GridSearchCV(estimator=Pipeline(steps=[('normalize', Normalizer()),
                                       ('est', SVR())]),
             param_grid={'est__C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02]),
                         'est__degree': [3, 4, 5],
                         'est__gamma': ['scale', 'auto'],
                         'est__kernel': ['rbf', 'linear', 'poly']},
             scoring='neg_mean_absolute_error')

In [49]:
print(gs_svr.best_params_)
best_svr = gs_svr.best_estimator_
y_preds = best_svr.predict(X_test)
mae = mean_absolute_error(y_test, y_preds)

print('mae:', mae)

{'est__C': 10.0, 'est__degree': 3, 'est__gamma': 'scale', 'est__kernel': 'poly'}
mae: 10.805507729346736


In [ ]:
X_train.columns

In [73]:
from sklearn.ensemble import RandomForestRegressor
max_features = list(np.linspace(0.1, 1, 10))
max_features.append('auto')
params = {'max_depth':[12], 
          'min_samples_split': [0.1],
          'min_samples_leaf':[0.1],
         'max_features': ['auto']}

gs_rf = GridSearchCV(RandomForestRegressor(), param_grid=params, scoring='neg_mean_absolute_error', verbose=2)

gs_rf.fit(X_train, y_train)

print(gs_rf.best_params_)
best_rf = gs_rf.best_estimator_
y_preds = best_rf.predict(X_test)
mae = mean_absolute_error(y_test, y_preds)

print('mae:', mae)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] max_depth=12, max_features=auto, min_samples_leaf=0.1, min_samples_split=0.1 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  max_depth=12, max_features=auto, min_samples_leaf=0.1, min_samples_split=0.1, total=   1.9s
[CV] max_depth=12, max_features=auto, min_samples_leaf=0.1, min_samples_split=0.1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.8s remaining:    0.0s


[CV]  max_depth=12, max_features=auto, min_samples_leaf=0.1, min_samples_split=0.1, total=   1.9s
[CV] max_depth=12, max_features=auto, min_samples_leaf=0.1, min_samples_split=0.1 
[CV]  max_depth=12, max_features=auto, min_samples_leaf=0.1, min_samples_split=0.1, total=   1.8s
[CV] max_depth=12, max_features=auto, min_samples_leaf=0.1, min_samples_split=0.1 
[CV]  max_depth=12, max_features=auto, min_samples_leaf=0.1, min_samples_split=0.1, total=   1.9s
[CV] max_depth=12, max_features=auto, min_samples_leaf=0.1, min_samples_split=0.1 
[CV]  max_depth=12, max_features=auto, min_samples_leaf=0.1, min_samples_split=0.1, total=   1.9s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    9.3s finished


{'max_depth': 12, 'max_features': 'auto', 'min_samples_leaf': 0.1, 'min_samples_split': 0.1}
mae: 10.72567300743439


## Saving the Model

In [74]:
import pickle

model_name = 'rf_model.sav'

with open(model_name, 'wb') as file:
    pickle.dump(best_rf, file)

In [75]:
with open(model_name, 'rb') as file:
    pickled_rf = pickle.load(file)
    
pickled_rf

RandomForestRegressor(max_depth=12, min_samples_leaf=0.1, min_samples_split=0.1)

## Using the Model

In [64]:
X_train.columns

Index(['HOME_MIN', 'HOME_FGM', 'HOME_FGA', 'HOME_FG3M', 'HOME_FG3A',
       'HOME_FTM', 'HOME_FTA', 'HOME_OREB', 'HOME_DREB', 'HOME_AST',
       'HOME_TOV', 'HOME_STL', 'HOME_BLK', 'HOME_BLKA', 'HOME_PF', 'HOME_PFD',
       'HOME_PTS', 'HOME_PLUS_MINUS', 'HOME_FGM_opp', 'HOME_FGA_opp',
       'HOME_FG3M_opp', 'HOME_FG3A_opp', 'HOME_FTM_opp', 'HOME_FTA_opp',
       'HOME_OREB_opp', 'HOME_DREB_opp', 'HOME_AST_opp', 'HOME_TOV_opp',
       'HOME_STL_opp', 'HOME_PTS_opp', 'HOME_WIN', 'HOME_prev_WIN',
       'HOME_prev2_WIN', 'AWAY_MIN', 'AWAY_FGM', 'AWAY_FGA', 'AWAY_FG3M',
       'AWAY_FG3A', 'AWAY_FTM', 'AWAY_FTA', 'AWAY_OREB', 'AWAY_DREB',
       'AWAY_AST', 'AWAY_TOV', 'AWAY_STL', 'AWAY_BLK', 'AWAY_BLKA', 'AWAY_PF',
       'AWAY_PFD', 'AWAY_PTS', 'AWAY_PLUS_MINUS', 'AWAY_FGM_opp',
       'AWAY_FGA_opp', 'AWAY_FG3M_opp', 'AWAY_FG3A_opp', 'AWAY_FTM_opp',
       'AWAY_FTA_opp', 'AWAY_OREB_opp', 'AWAY_DREB_opp', 'AWAY_AST_opp',
       'AWAY_TOV_opp', 'AWAY_STL_opp', 'AWAY_PTS_opp', 'AWAY_WIN

In [65]:
from nba_api.stats.endpoints import teamgamelogs

def prep_inputs_for_model(homeTeamAbbr, awayTeamAbbr, season='2020-21'):
    
    gamelogs = teamgamelogs.TeamGameLogs(season_nullable=season).get_data_frames()[0]
    gamelogs_clean = clean_team_gamelog(gamelogs)
        
    home_gl = gamelogs_clean.loc[gamelogs_clean['TEAM_ABBREVIATION'] == homeTeamAbbr]

    away_gl = gamelogs_clean.loc[gamelogs_clean['TEAM_ABBREVIATION'] == awayTeamAbbr]

    home_stats_weighted = get_weighted_cume_stats(home_gl)
    home_stats_last = home_stats_weighted.iloc[-1].to_frame().T

    away_stats_weighted = get_weighted_cume_stats(away_gl)
    away_stats_last = away_stats_weighted.iloc[-1].to_frame().T
    
    inputs = pd.merge(home_stats_last, away_stats_last, on='SEASON_YEAR')
    # For readability, rename columns    
    new_col_names = ['HOME_' + x[:-2] if x[-2:] == '_x' else 'AWAY_' + x[:-2] for x in inputs.columns.to_list()]
    inputs.columns = new_col_names
    
    columns_needed = ['HOME_MIN', 'HOME_FGM', 'HOME_FGA', 'HOME_FG3M', 'HOME_FG3A',
       'HOME_FTM', 'HOME_FTA', 'HOME_OREB', 'HOME_DREB', 'HOME_AST',
       'HOME_TOV', 'HOME_STL', 'HOME_BLK', 'HOME_BLKA', 'HOME_PF', 'HOME_PFD',
       'HOME_PTS', 'HOME_PLUS_MINUS', 'HOME_FGM_opp', 'HOME_FGA_opp',
       'HOME_FG3M_opp', 'HOME_FG3A_opp', 'HOME_FTM_opp', 'HOME_FTA_opp',
       'HOME_OREB_opp', 'HOME_DREB_opp', 'HOME_AST_opp', 'HOME_TOV_opp',
       'HOME_STL_opp', 'HOME_PTS_opp', 'HOME_WIN', 'HOME_prev_WIN',
       'HOME_prev2_WIN', 'AWAY_MIN', 'AWAY_FGM', 'AWAY_FGA', 'AWAY_FG3M',
       'AWAY_FG3A', 'AWAY_FTM', 'AWAY_FTA', 'AWAY_OREB', 'AWAY_DREB',
       'AWAY_AST', 'AWAY_TOV', 'AWAY_STL', 'AWAY_BLK', 'AWAY_BLKA', 'AWAY_PF',
       'AWAY_PFD', 'AWAY_PTS', 'AWAY_PLUS_MINUS', 'AWAY_FGM_opp',
       'AWAY_FGA_opp', 'AWAY_FG3M_opp', 'AWAY_FG3A_opp', 'AWAY_FTM_opp',
       'AWAY_FTA_opp', 'AWAY_OREB_opp', 'AWAY_DREB_opp', 'AWAY_AST_opp',
       'AWAY_TOV_opp', 'AWAY_STL_opp', 'AWAY_PTS_opp', 'AWAY_WIN',
       'AWAY_prev_WIN', 'AWAY_prev2_WIN']
    inputs = inputs[columns_needed]
    
    return inputs

In [66]:
prep_inputs_for_model('LAC', 'OKC', season='2020-21')

,HOME_MIN,HOME_FGM,HOME_FGA,HOME_FG3M,HOME_FG3A,HOME_FTM,HOME_FTA,HOME_OREB,HOME_DREB,HOME_AST,HOME_TOV,HOME_STL,HOME_BLK,HOME_BLKA,HOME_PF,HOME_PFD,HOME_PTS,HOME_PLUS_MINUS,HOME_FGM_opp,HOME_FGA_opp,HOME_FG3M_opp,HOME_FG3A_opp,HOME_FTM_opp,HOME_FTA_opp,HOME_OREB_opp,HOME_DREB_opp,HOME_AST_opp,HOME_TOV_opp,HOME_STL_opp,HOME_PTS_opp,HOME_WIN,HOME_prev_WIN,HOME_prev2_WIN,AWAY_MIN,AWAY_FGM,AWAY_FGA,AWAY_FG3M,AWAY_FG3A,AWAY_FTM,AWAY_FTA,AWAY_OREB,AWAY_DREB,AWAY_AST,AWAY_TOV,AWAY_STL,AWAY_BLK,AWAY_BLKA,AWAY_PF,AWAY_PFD,AWAY_PTS,AWAY_PLUS_MINUS,AWAY_FGM_opp,AWAY_FGA_opp,AWAY_FG3M_opp,AWAY_FG3A_opp,AWAY_FTM_opp,AWAY_FTA_opp,AWAY_OREB_opp,AWAY_DREB_opp,AWAY_AST_opp,AWAY_TOV_opp,AWAY_STL_opp,AWAY_PTS_opp,AWAY_WIN,AWAY_prev_WIN,AWAY_prev2_WIN
0,48,41.5,88.5,14.1,36.45,17,19.3,9.8,37.85,24.35,12.25,9.05,4.55,3.1,16.9,16.8,114.1,12.5,38.45,87.05,11.2,32.2,13.5,16.8,8.45,34.45,22.65,15,6.95,101.6,0.85,1,1,48.5,40.85,88.95,13.4,38.75,14,20.45,7.65,36.95,22.5,12.6,6.95,4.6,4.15,19.5,18.4,109.1,-3.45,41.7,94.05,14,40.4,15.15,18.75,10.65,39.15,22.65,12.85,6.55,112.55,0.55,1,1


In [67]:
predictions_df = pd.DataFrame({'date':[], 'home_team':[], 'away_team':[], 'prediction':[], 'result':[]})
predictions_df

,date,home_team,away_team,prediction,result


In [68]:
from datetime import datetime
from nba_api.stats.endpoints import scoreboardv2



def get_matchups(date = datetime.today().strftime('%Y-%m-%d')):
    scoreboard_df = scoreboardv2.ScoreboardV2(game_date=datetime.today().strftime("%Y-%m-%d")).get_data_frames()[0]
    teams = scoreboard_df['GAMECODE'].str[-6:]
    away_teams = teams.str[:3]
    home_teams = teams.str[3:]
    matchups = tuple(zip(home_teams, away_teams))
    return matchups

get_matchups()
    
    

(('ATL', 'LAL'),
 ('MIA', 'CHA'),
 ('CLE', 'MIN'),
 ('CHI', 'NYK'),
 ('MIL', 'POR'),
 ('NOP', 'SAC'),
 ('OKC', 'HOU'),
 ('DAL', 'PHX'),
 ('SAS', 'MEM'),
 ('DEN', 'DET'))

In [76]:
matchups = get_matchups()
todays_preds = pd.DataFrame({'model', 'date':[], 'home_team':[], 'away_team':[], 'prediction':[], 'result':[]})
game_date = []
home_team = []
away_team = []
game_prediction = []
result = []
for matchup in matchups:
    date = datetime.today().strftime('%Y-%m-%d')
    inputs = prep_inputs_for_model(matchup[0], matchup[1])
    prediction = round(pickled_rf.predict(inputs)[0], 1)
    game_date.append(date)
    home_team.append(matchup[0])
    away_team.append(matchup[1])
    game_prediction.append(prediction)
    result.append(np.nan)
    row = pd.DataFrame({'date':[date], 'home_team':[matchup[0]], 'away_team':[matchup[1]], 'prediction':[prediction], 'result':[np.nan]})
    todays_preds = pd.concat([todays_preds, row])

todays_preds
    
predictions_df = predictions_df.append(todays_preds)

predictions_df

,date,home_team,away_team,prediction,result
0,2021-01-26,ATL,LAL,1.7,NaN
0,2021-01-26,MIA,CHA,-1.6,NaN
0,2021-01-26,CLE,MIN,3.6,NaN
0,2021-01-26,CHI,NYK,5.8,NaN
0,2021-01-26,MIL,POR,6.9,NaN
0,2021-01-26,NOP,SAC,3.2,NaN
0,2021-01-26,OKC,HOU,-0.8,NaN
0,2021-01-26,DAL,PHX,-2.7,NaN
0,2021-01-26,SAS,MEM,3.8,NaN
0,2021-01-26,DEN,DET,7.6,NaN


In [71]:
predictions_df.to_csv('predictions.csv', index=False)

In [164]:
todays_preds

,date,home_team,away_team,prediction,result
0,2021-01-26,CLE,DET,-0.4,NaN
0,2021-01-26,ORL,SAC,4.0,NaN
0,2021-01-26,TOR,MIL,4.7,NaN
0,2021-01-26,CHA,IND,3.7,NaN
0,2021-01-26,PHI,LAL,-0.2,NaN
0,2021-01-26,ATL,BKN,1.1,NaN
0,2021-01-26,DEN,MIA,6.2,NaN
0,2021-01-26,SAS,BOS,1.9,NaN
0,2021-01-26,UTA,DAL,11.8,NaN
